In [1]:
from bz2 import decompress
from concurrent.futures import thread
from filecmp import BUFSIZE
from re import A
import socket
import numpy as np
import cv2
import time
import zlib
import os
import time
import threading
import numpy as np
import cv2
import os
import matplotlib as plt
import math
import utm
import time
from pyaos.LFR_utils import read_poses_and_images,pose_to_virtualcamera, init_aos, init_window
import pyaos.LFR_utils as utils
from pathlib import Path
import pyaos.lfr as LFR
from PIL import Image
import glm
from datetime import datetime
import importlib
import copy
import numpy as np
import matplotlib.pyplot as plt
import time
from matplotlib.animation import FuncAnimation
from scipy import interpolate
import spectral
from skimage import img_as_float
import utm
import cv2
import math
import glm
import scipy.interpolate
import time
import os
from zipfile import ZipFile
import glob
import shutil

from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.chrome import ChromeDriverManager

ModuleNotFoundError: No module named 'pyaos.LFR_utils'

In [ ]:
Download_Location = r'C:\Users\Rakesh\Downloads'
print(Download_Location)
Images_Path = os.path.join(Download_Location,'images')
os.mkdir(Images_Path)
Stages_Path = os.path.join(Download_Location,'stages')
os.mkdir(Stages_Path)
Integral_Path = os.path.join(Download_Location,'integrals')
os.mkdir(Integral_Path)
Live_Debug_Path = os.path.join(Download_Location,'live_debug')
os.mkdir(Live_Debug_Path)
Final_result = os.path.join(Download_Location,'Final_result')
os.mkdir(Final_result)
plots = os.path.join(Download_Location,'plots')
os.mkdir(plots)
leader_blob_image_folder = os.path.join(Download_Location,'leader_blob_image')
os.mkdir(leader_blob_image_folder)
leader_rx_image_folder = os.path.join(Download_Location,'leader_rx_image')
os.mkdir(leader_rx_image_folder)
gen_integral_blob_img_folder = os.path.join(Download_Location,'gen_integral_blob_img')
os.mkdir(gen_integral_blob_img_folder)




In [ ]:
# import time
# import os
# from zipfile import ZipFile
# import json
# import os
  
# path = r'C:\Users\Rakesh\Downloads'  # path where the tree.json file shpould be kept

# camera_info_file = os.path.join(path,'trees_500.json')
# with open(camera_info_file) as json_file:
#     camera_Dict = json.load(json_file)
# print(len(camera_Dict['positions']))
# xaxislist = []
# zaxislist = []
# yaxislist = []
# for i in range (0, len(camera_Dict['positions'] )):
#     xaxislist.append(camera_Dict['positions'][i]['x'])
#     yaxislist.append(camera_Dict['positions'][i]['y'])
#     zaxislist.append(camera_Dict['positions'][i]['z'])
    
# treex = ' '.join([str(item) for item in xaxislist])
# treey = ' '.join([str(item) for item in zaxislist])
treex = str(0)
treey = str(0)

#print(zaxislist)



In [ ]:


class evaluate_metric:
    _aos = None
    _fov = 22.815436217896945#43.10803984095769#43.50668199945787#50.815436217896945
    _rx_threshold = 0.999
    _debug = False
    _x_off = 0.0
    _y_off = 0.0
    _f_ = None
    _focalplane_height = 0.0
    _compass_correction = 0.0
    _integration_count = 0
    _previous_img_mask = None
    _previous_detection_pose = None
    _prev_images = []
    _prev_thermal_images = []
    _current_thermal_images_list = []
    _current_RGB_images_list = []
    _current_center_camera_id = None
    
    _calc_leader_current_integral = True

    _prev_images_poses = []
    _prev_blob_images = []

    def __init__(self,aos_ptr,fieldofview,rx_threshold, dem_height, compasscorrection, debug_mode = False):
        self._aos = aos_ptr
        self._fov = fieldofview
        self._rx_threshold = rx_threshold
        self._focalplane_height = dem_height
        self._compass_correction = compasscorrection
        self._debug = debug_mode
        self._integration_count = 0
        if self._debug:
            # build random function
            xc, yc = self.get_mesh()
            #np.random.seed(100)#int(time.time())
            np.random.seed(int(time.time()))
            z = np.random.rand(*xc.shape)*30-15 # -15...15
            self._x_off = self._y_off = 0.0
            # define the objective function
            self._f_ = interpolate.RBFInterpolator(self.to_linear(xc, yc), z.ravel())

    def set_rx_threshold(self, rx_threshold):
        self._rx_threshold = rx_threshold

    def get_rx_threshold(self):
        return self._rx_threshold
    
    def set_focalplane_height(self, dem_height):
        self._focalplane_height = dem_height

    def get_focalplane_height(self):
        return self._focalplane_height
    
    def set_fov(self, fieldofview):
        self._fov = fieldofview

    def get_fov(self):
        return self._fov
    
    def set_compass_correction(self, compasscorrection):
        self._compass_correction = compasscorrection

    def get_compass_correction(self):
        return self._compass_correction
    


    def f(self, x,y):
        #global x_off,y_off
        s=np.reshape(self._f_(np.stack([x.ravel(),y.ravel()], -1)), x.shape)
        m=np.sin((x/50)* (3.14/2))*np.sin((y/50)* (3.14/2))
        m[m < 0.8] = 0
        m[m > 0.8] = 1
        #m=np.sin(x+self._x_off)*np.sin(y+self._y_off)
        #m[m < 0.8] = 0
        #m[m > 0.8] = 1
        s=s*m
        s[s == 0] = -15
        return s     
    def get_mesh(self,n=20): # meshsize
        return np.meshgrid(np.linspace(0, 100, n),
                       np.linspace(0, 100, n + 1))

    def to_linear(self,x, y): 
        return np.stack([x.ravel(), y.ravel()], -1)

    def divide_by_alpha(self,rimg2):
        a = np.stack((rimg2[:,:,3],rimg2[:,:,3],rimg2[:,:,3]),axis=-1)
        return rimg2[:,:,:3]/a

    def pose_to_virtualcamera(self, vpose ):
        vp = glm.mat4(*np.array(vpose).transpose().flatten())
        #vp = vpose.copy()
        ivp = glm.inverse(glm.transpose(vp))
        #ivp = glm.inverse(vpose)
        Posvec = glm.vec3(ivp[3])
        Upvec = glm.vec3(ivp[1])
        FrontVec = glm.vec3(ivp[2])
        lookAt = glm.lookAt(Posvec, Posvec + FrontVec, Upvec)
        cameraviewarr = np.asarray(lookAt)
        #print(cameraviewarr)
        return cameraviewarr
        # #vp = glm.mat4(*np.array(vpose).transpose().flatten())
        # inversecamerapose = glm.inverse(vpose)
        # #ivp = glm.inverse(glm.transpose(inversecamerapose))
        # Posvec = glm.vec3(inversecamerapose[3])
        # Upvec = glm.vec3(inversecamerapose[1])
        # FrontVec = glm.vec3(inversecamerapose[2])
        # cameraviewarr = np.array(glm.lookAt(Posvec, Posvec + FrontVec, Upvec)) 
        # return cameraviewarr
        # #lookAt = glm.lookAt(Posvec, Posvec + FrontVec, Upvec)
        # #return np.asarray(glm.transpose(lookAt))
   

    def detect_anomaly(self,single_images, confCoefficient):
        bw_images_list = []
        rx_images_list = []
        print("confCoefficient", confCoefficient)
        for i in range(len(single_images)):
            RGB_image = single_images[i]
            rxScore = spectral.rx(RGB_image)
            min_rx_score = np.min(rxScore)
            max_rx_score = np.max(rxScore)
            rxScore = img_as_float(rxScore)
            rescaled_rx_score = ((rxScore - min_rx_score) / (max_rx_score - min_rx_score))
            rescaled_rx_score_int = np.asarray(rescaled_rx_score*255,dtype=np.uint8)
            #print('Normalized Max Score:', np.min(rescaled_rx_score_int))
            #print('Normalized Min Score:', np.max(rescaled_rx_score_int))
            count, bins_count = np.histogram(rescaled_rx_score_int, bins = 256)
            # finding the PDF of the histogram using count values
            pdf = count / np.prod(np.size(rescaled_rx_score_int)) 
            # using numpy np.cumsum to calculate the CDF
            # We can also find using the PDF values by looping and adding
            cdf = np.cumsum(pdf)
            rxThreshold = [ n for n,i in enumerate(cdf) if i>confCoefficient ][1]
            ##rxThreshold =  np.nonzero(cdf > confCoefficient, 1 );
            print ('RX Threshold: ' , rxThreshold)
            rx_images_list.append(rxScore)
            bw = rescaled_rx_score_int > rxThreshold
            cv_bw = np.asarray(bw*255,dtype=np.uint8)
            img = np.zeros((cv_bw.shape[0],cv_bw.shape[1],3),dtype=RGB_image.dtype)
            img[:,:,0] = cv_bw
            img[:,:,1] = cv_bw
            img[:,:,2] = cv_bw
            bw_images_list.append(img)
            #cv2.imwrite(os.path.join( Live_Debug_Path,  str(self._integration_count)+ '_' + str(i)+'_rx_blob.png'), img)
            #nzCount = np.count_nonzero(bw)
            #print('Total NZ Count after Threshold: ', nzCount)
        return bw_images_list,rx_images_list
    
    def project_images_to_all(self,aos, img_list, pose_list,fov,center_camera_index = None, project_images = False):
        aos.clearViews()
        proj_img_list = []
        for i in range(len(img_list)):
            aos.addView(img_list[i], pose_list[i], "DEM BlobTrack")
        aos.setDEMTransform([0,0,self._focalplane_height])
        #aos.setPoseCorrectionall(len(img_list),[0,0,0], [0,0, self._compass_correction])
        for i in range(len(img_list)):
            renderering_ids = []
            if project_images:
                renderering_ids.append(i)
                if center_camera_index == None:
                    for j in range(len(img_list)):
                        cmr_proj = []
                        img_renderering_ids = []
                        img_renderering_ids.append(j)
                        #proj_img = aos.render(self.pose_to_virtualcamera(aos.getPose(i)), fov, img_renderering_ids)
                        proj_img = aos.render(self.pose_to_virtualcamera(pose_list[i]), fov, img_renderering_ids)
                        tmp = self.divide_by_alpha(proj_img)
                        cmr_proj.append(tmp)
                    proj_img_list.append(cmr_proj)
                else:
                    proj_img = aos.render(self.pose_to_virtualcamera(pose_list[center_camera_index]), fov, renderering_ids)
                    tmp = self.divide_by_alpha(proj_img)
                    proj_img_list.append(tmp)
            else:
                if center_camera_index == None:
                    proj_img = aos.render(self.pose_to_virtualcamera(pose_list[i]), fov, renderering_ids)
                    tmp = self.divide_by_alpha(proj_img)
                    proj_img_list.append(tmp)
                else:
                    if i == center_camera_index:
                        proj_img = aos.render(self.pose_to_virtualcamera(pose_list[center_camera_index]), fov, renderering_ids)
                        tmp = self.divide_by_alpha(proj_img)
                        proj_img_list.append(tmp)
        return proj_img_list

    def find_center_camera_highest_projection(self,image_list):
        max_contour_img_id = None
        max_countour_list = []
        max_countour_center_list = []
        max_contour_bounding_rect_list = []
        contour_img = np.zeros((image_list[0].shape[0],image_list[0].shape[1]))
        ################For all Images calculate contours and find max###########################################
        #######################contour in each images######################################################
        for i in range(len(image_list)):
            contours,hierarchy  = cv2.findContours(np.asarray(image_list[i][:,:,0],dtype=np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            max_countour = 0
            max_countour_center = ()
            max_contour_bounding_rect = ()
            #print(i,len(contours))
            for k in range(len(contours)):
                contour_area = cv2.contourArea(contours[k])
                x,y,w,h = cv2.boundingRect(contours[k])
                #contour_area_list.append(contour_area)
                if contour_area > max_countour:
                    max_countour = contour_area
                    M = cv2.moments(contours[k])
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    max_countour_center = (cX,cY)
                    max_contour_bounding_rect = (x,y,w,h)
            max_countour_list.append(max_countour)
            max_countour_center_list.append(max_countour_center)
            max_contour_bounding_rect_list.append(max_contour_bounding_rect)
        ################For Image Indices with Highest Contour###########################################
        #################If Two Image has same Contour length######################################################
        #################than select one where blob is closer to center######################################################
        print("Integral max_countour_list", max_countour_list)
        print("Integral max_countour_list", max_countour_center_list)
        max_indices = [index for index, item in enumerate(max_countour_list) if item == max(max_countour_list)]
        if len(max_indices) > 1:
            distance_center = 10000000
            for i in range(len(max_indices)):
                distance = math.sqrt(math.pow((max_countour_center_list[i][0] - 512),2)+ math.pow((max_countour_center_list[i][1] - 512),2))
                if distance < distance_center:
                    max_contour_img_id = max_indices[i]
                    distance_center  = distance
        else:
            max_contour_img_id = max_indices[0]
        ################Get Bounding box location of contour and create###########################################
        ######################mask image with rect set to 255######################################################
        rect = max_contour_bounding_rect_list[max_contour_img_id]
        print("max_contour_img_id", max_contour_img_id)
        print("rect of bounding box", rect)
        contour_img[rect[1]:rect[1]+rect[3],rect[0]:rect[0]+rect[2]] = 255
        center_img_max_contour  = max_countour_list[max_contour_img_id]
        ################return center image with highest contour and ###########################################
        ######################its corresponding mask image and ######################################################
        ######################its corresponding contour area######################################################
        return max_contour_img_id, contour_img, center_img_max_contour,rect
    
    def find_leader(self,single_proj_img, mask_img):
        max_contour = 0
        leader_id = None
        ################For all Images calculate contours within the mask###########################################
        #######################and find image with maximum contour######################################################
        for i in range(len(single_proj_img)):
            img = single_proj_img[i]
            tmp_img = np.asarray(img[:,:,0],dtype=np.uint8)
            tmp_mask_img = np.asarray(mask_img,dtype=np.uint8)
            #print("sizes", tmp_img.shape, tmp_mask_img.shape)
            #print("dtype", tmp_img.dtype, tmp_mask_img.dtype)
            #tmp_img = np.zeros((single_proj_img[i].shape[0],single_proj_img[i].shape[1]),dtype=np.uint8)
            cnt_tmp_img = np.zeros((single_proj_img[i].shape[0],single_proj_img[i].shape[1]),dtype=np.uint8)
            cv2.bitwise_and(tmp_img, tmp_mask_img, cnt_tmp_img)
            contours,hierarchy  = cv2.findContours(np.asarray(cnt_tmp_img[:,:],dtype=np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            curr_img_max_contour = 0
            for k in range(len(contours)):
                contour_area = cv2.contourArea(contours[k])
                if contour_area > curr_img_max_contour:
                    curr_img_max_contour = contour_area
            print("Single curr_img_max_contour", i, curr_img_max_contour)
            if curr_img_max_contour > max_contour:
                max_contour = curr_img_max_contour
                leader_id = i
        ################return leader index and maximum contour size within the leader###########################################
        #######################and find image with maximum contour######################################################
        return leader_id, max_contour
    
    def find_leader_within_images(self,single_images,site_poses, fov = None, rxthreshold =  None, dem_height = None, compasscorrection = None ,pos = None):
        if fov == None:
            fov = self._fov
        else:
            self._fov = fov
        if rxthreshold == None:
            rxthreshold = self._rx_threshold
        else:
            self._rx_threshold = rxthreshold
        if dem_height == None:
            dem_height = self._focalplane_height
        else :
            self._focalplane_height = dem_height
        if compasscorrection == None:
            compasscorrection = self._compass_correction
        else :
            self._compass_correction = compasscorrection
        if self._debug:
            obj = self.f(pos[0], pos[1])
            Leader_index = obj.argmax()
            leader_contour = obj[Leader_index]
        else :
            self._current_thermal_images_list = []
            self._current_RGB_images_list = []
            if len(single_images)>=1:
                no_channels = single_images[0].shape[2]
                if no_channels > 3:
                    for i in range(len(single_images)):
                        img = single_images[i]
                        rgb_img = img[:,:,0:3]
                        thermal_img = np.zeros(rgb_img.shape,dtype = rgb_img.dtype)
                        thermal_img[:,:,0] = img[:,:,3]
                        thermal_img[:,:,1] = img[:,:,3]
                        thermal_img[:,:,2] = img[:,:,3]
                        self._current_RGB_images_list.append(rgb_img)
                        self._current_thermal_images_list.append(thermal_img)
                elif no_channels == 3:
                    for i in range(len(single_images)):
                        img = single_images[i]
                        self._current_RGB_images_list.append(rgb_img)
  
            rgb_integral_images = self.project_images_to_all(self._aos, self._current_RGB_images_list, site_poses,fov,center_camera_index=None,project_images=False)
            if len(self._current_thermal_images_list)>= 1:
                thermal_integral_images = self.project_images_to_all(self._aos, self._current_thermal_images_list, site_poses,fov,center_camera_index=None,project_images=False)
            ################################################################
            ##############After Generating Blob Images############################
            #########Set Each camera as virtual camera and ####################
            ##############project all Rx_bw images###########################
            #print("len of single images", len(single_images))
            #########First do Blob detection on all Single Images###########
            blob_images,rx_images = self.detect_anomaly(single_images, confCoefficient = rxthreshold)
            ###############REquired Images##########################################
            tmp_blob_images = blob_images
            tmp_rx_images = rx_images
            #display_all(single_images) //---- display all Single Images in subplots
            #display_all(blob_images) //---- display all bw Images in subplots
            ################################################################
            ##############After Generating Blob Images############################
            #########Set Each camera as virtual camera and ####################
            ##############project all Rx_bw images###########################
            proj_images = self.project_images_to_all(self._aos, blob_images, site_poses,fov,center_camera_index=None,project_images=False)
            #print(len(proj_images))  //debug
            ################################################################
            ##############After Generating integrated Blob Images############################
            #########Find Which Image contains the Maximum Contour ####################
            ##########and generate a Mask Image with the maximum contour###########################
            center_camera_id, mask_img, center_camera_max_contour_area,rect = self.find_center_camera_highest_projection(proj_images)
            self._current_center_camera_id = center_camera_id
            #print(center_camera_id)  //debug
            if self._calc_leader_current_integral == False:
                #######################################################################
                ##############After Finding the Center Camera############################
                #########Find Projection of All images to this Camera####################
                ########################################################################
                single_proj_images = self.project_images_to_all(self._aos, blob_images, site_poses,fov,center_camera_index=center_camera_id,project_images=True)
                ################################################################
                ##############Now Find Leader############################
                #########By Finding largest Contour in this Image####################
                ###################################################################
                Leader_index, leader_contour = self.find_leader(single_proj_images,mask_img)
                leader_blob_image = tmp_blob_images[Leader_index]
            else:
                Leader_index = center_camera_id
                leader_contour = center_camera_max_contour_area
                leader_blob_image = proj_images[Leader_index]
            
            leader_rx_image = tmp_rx_images[Leader_index]
            print("leader_rx_image datatype",leader_rx_image.dtype)
            print("leader_rx_image dimensions",leader_rx_image.shape)
            
            #Leader_index = 1
            #center_camera_id = 1
            center_rx_integral_image  = np.asarray(proj_images[center_camera_id],dtype=np.uint8)
            center_rgb_integral_image  = np.asarray(rgb_integral_images[center_camera_id],dtype=np.uint8)
            #leader_rx_image_proj = np.asarray(single_proj_images[Leader_index],dtype=np.uint8)
            leader_unprojected_rx_image = np.asarray(blob_images[Leader_index],dtype=np.uint8)
            center_thermal_integral_image = np.zeros((single_images[0].shape[0],single_images[0].shape[1],single_images[0].shape[2]),dtype=np.uint8)
            if len(self._current_thermal_images_list)>= 1:
                center_thermal_integral_image = np.asarray(thermal_integral_images[center_camera_id],dtype=np.uint8)
                cv2.imwrite(os.path.join( Live_Debug_Path,  str(self._integration_count)+ '_thermal_integral.png'), center_thermal_integral_image)
                cv2.imwrite(os.path.join( Integral_Path,  str(self._integration_count)+ '_thermal_integral.png'), center_thermal_integral_image)
            #datasets_folder_live_debug = os.path.join( r"E:\Droneswarms_Feb_clientserverdevelopment\live_debug" )
            cv2.imwrite(os.path.join( Live_Debug_Path,  str(self._integration_count)+'_'+str(int(Leader_index))+'_'+str(rxthreshold)+ '_rxunprojected.png'), leader_unprojected_rx_image); 
            #cv2.imwrite(os.path.join( Live_Debug_Path,  str(self._integration_count)+'_'+str(int(Leader_index))+'_'+str(int(leader_contour))+ '_rx.png'), leader_rx_image_proj); 
            cv2.imwrite(os.path.join( Live_Debug_Path,  str(self._integration_count)+ '_integral.png'), center_rgb_integral_image)
            cv2.imwrite(os.path.join( Live_Debug_Path,  str(self._integration_count)+'_'+str(int(center_camera_id))+'_'+str(int(center_camera_max_contour_area))+ '_rxintegral.png'), center_rx_integral_image)
            cv2.imwrite(os.path.join( Integral_Path,  str(self._integration_count)+ '_integral.png'), center_rgb_integral_image)
            print('Image integral Dimensions :', center_rgb_integral_image.shape);
            #cv2.imwrite(os.path.join( Integral_Path,  str(self._integration_count)+'_'+str(int(center_camera_id))+'_'+str(int(center_camera_max_contour_area))+ '_rxintegral.png'), center_rx_integral_image)
            center_mask_img = np.asarray(mask_img,dtype=np.uint8)
            cv2.imwrite(os.path.join( Live_Debug_Path,  str(self._integration_count)+'_'+str(int(Leader_index))+'_'+str(int(leader_contour))+ '_rxintegralmask.png'), center_mask_img)
        return Leader_index, leader_contour, center_mask_img, center_camera_id,leader_blob_image, leader_rx_image,rect
            
        #     center_rgb_integral_image  = np.asarray(rgb_integral_images[center_camera_id],dtype=np.uint8)
        #     leader_rx_image = np.asarray(single_proj_images[Leader_index],dtype=np.uint8)
        #     datasets_folder_drone2 = os.path.join( r"E:\Droneswarms_Feb_clientserverdevelopment\drone2" )
        #     cv2.imwrite(os.path.join( datasets_folder_drone2,  str(self._integration_count)+ '_rx.jpg'), leader_rx_image); 
        #     cv2.imwrite(os.path.join( datasets_folder_drone2,  str(self._integration_count)+ '_integral.jpg'), center_rgb_integral_image);
        #     self._integration_count = self._integration_count + 1
        #     center_mask_img = np.asarray(mask_img,dtype=np.uint8)
        # return Leader_index, leader_contour, center_mask_img, center_camera_id
    
    def get_relevent_info(self,single_images,site_poses, fov = None, rxthreshold =  None, dem_height = None, compasscorrection = None , pos = None):
        if fov == None:
            fov = self._fov
        else:
            self._fov = fov
        if rxthreshold == None:
            rxthreshold = self._rx_threshold
        else:
            self._rx_threshold = rxthreshold
        if dem_height == None:
            dem_height = self._focalplane_height
        else :
            self._focalplane_height = dem_height
        if compasscorrection == None:
            compasscorrection = self._compass_correction
        else :
            self._compass_correction = compasscorrection
        ################################################################
        ##############After Generating Blob Images############################
        #########Set Each camera as virtual camera and ####################
        ##############project all Rx_bw images###########################
        print("len of single images", len(single_images))
        rgb_integral_images = self.project_images_to_all(self._aos, single_images, site_poses,fov,center_camera_index=None,project_images=False)
        #########First do Blob detection on all Single Images###########
        blob_images,rx_images = self.detect_anomaly(single_images, confCoefficient = rxthreshold)
        print("len of blob images", len(blob_images))
        for i in range(len(blob_images)):
            cv2.imwrite(os.path.join( Live_Debug_Path,  str(i)+ '_blobimages_debug_blobthreshold.png'), np.asarray(blob_images[i],dtype=np.uint8))
        #display_all(single_images) //---- display all Single Images in subplots
        #display_all(blob_images) //---- display all bw Images in subplots
        ################################################################
        ##############After Generating Blob Images############################
        #########Set Each camera as virtual camera and ####################
        ##############project all Rx_bw images###########################
        proj_images = self.project_images_to_all(self._aos, blob_images, site_poses,fov,center_camera_index=None,project_images=False)
        print("len of proj images", len(proj_images))
        for i in range(len(proj_images)):
            cv2.imwrite(os.path.join( Live_Debug_Path,  str(i)+ '_integral_debug_blobthreshold.png'), np.asarray(proj_images[i],dtype=np.uint8))
        #print(len(proj_images))  //debug
        ################################################################
        ##############After Generating integrated Blob Images############################
        #########Find Which Image contains the Maximum Contour ####################
        ##########and generate a Mask Image with the maximum contour###########################
        center_camera_id, mask_img, center_camera_max_contour_area,rect = self.find_center_camera_highest_projection(proj_images)
        #print(center_camera_id)  //debug
        if self._calc_leader_current_integral == False:
            #######################################################################
            ##############After Finding the Center Camera############################
            #########Find Projection of All images to this Camera####################
            ########################################################################
            single_proj_images = self.project_images_to_all(self._aos, blob_images, site_poses,fov,center_camera_index=center_camera_id,project_images=True)
            ################################################################
            ##############Now Find Leader############################
            #########By Finding largest Contour in this Image####################
            ###################################################################
            Leader_index, leader_contour = self.find_leader(single_proj_images,mask_img)
            leader_rx_image = np.asarray(single_proj_images[Leader_index],dtype=np.uint8)
        else:
            Leader_index = center_camera_id
            leader_contour = center_camera_max_contour_area
            leader_rx_image = np.asarray(proj_images[Leader_index],dtype=np.uint8)


        center_rgb_integral_image  = np.asarray(rgb_integral_images[center_camera_id],dtype=np.uint8)
        
        return center_rgb_integral_image, leader_rx_image, Leader_index, leader_contour
        
    
    def should_add_image_integral(self,aos,integral_img,single_img,img_poses,fov = 22.815436217896945,rx_threshold = 0.998):
        #########First do Blob detection on all Integral Image###########
        blob_images,rx_images = self.detect_anomaly(integral_img, confCoefficient = rx_threshold)
        #display_all(single_images) //---- display all Single Images in subplots
        #display_all(blob_images) //---- display all bw Images in subplots
        ################################################################
        ##############After Generating integrated Blob Images############################
        #########Find Which Image contains the Maximum Contour ####################
        ##########and generate a Mask Image with the maximum contour###########################
        center_camera_id, mask_img, center_camera_max_contour_area,rect = self.find_center_camera_highest_projection(blob_images)
        #print(center_camera_id)  //debug
        future_integaral_img_list = []
        future_integaral_img_list.append(integral_img)
        future_integaral_img_list.append(single_img)
        #######################################################################
        ##############After Finding the Center Camera############################
        #########Find Projection of All images to this Camera####################
        ########################################################################
        future_integaral_img = self.project_images_to_all(self._aos, future_integaral_img_list, img_poses,fov,center_camera_index=0,project_images=False)
        ################################################################
        ##############Now Find Leader############################
        #########By Finding largest Contour in this Image####################
        ###################################################################
        center_camera_id, mask_img, future_integaral_img_max_contour_area = self.find_center_camera_highest_projection(future_integaral_img)
        if (center_camera_max_contour_area >= future_integaral_img_max_contour_area ):
            return False
        else:
            return True
            
    def integrate_informative_images(self, single_images, single_poses, virtual_camera_pose, detection_mask = None, detection_pose = None):
        ############generate blob images --- can be optimized later to remove anomaly detection again and again####################################
        blob_images,rx_images = self.detect_anomaly(single_images, confCoefficient = self._rx_threshold)
        self._aos.clearViews()
        prev_img_max_contour = 0
        gen_integral_blob_img = np.zeros((blob_images[0].shape[0],blob_images[0].shape[1]),dtype=np.uint8)
        ##########################Check if previous mask is there if not if there is new mask use it########################################
        if detection_mask is None:
            current_detection_mask = self._previous_img_mask
            current_detection_pose = self._previous_detection_pose
        else:
            current_detection_mask = detection_mask
            current_detection_pose = detection_pose
            self._previous_img_mask = detection_mask
            self._previous_detection_pose = detection_pose
        if type(current_detection_mask) is np.ndarray :
            ##########################Project the mask to current virtual camera########################################
            self._aos.addView(current_detection_mask, current_detection_pose, "Detection Mask")
            img_renderering_ids = []
            self._aos.setDEMTransform([0,0,self._focalplane_height])
            #aos.setPoseCorrectionall(1,[0,0,0], [0,0, self._compass_correction])
            tmp_projected_detection_mask = self._aos.render(self.pose_to_virtualcamera(single_poses[self._current_center_camera_id]), self._fov, img_renderering_ids)
            projected_detection_mask = self.divide_by_alpha(tmp_projected_detection_mask)
            self._aos.clearViews()
            prev_img_max_contour = 0
            ##########################If previous images exist than find best contour of prev integral using previous blob images########################################
            if len(self._prev_images) != 0:
                for i in range(len(self._prev_images)):
                    self._aos.addView(self._prev_blob_images[i], self._prev_images_poses[i], "DEM BlobTrack")
                img_renderering_ids = []
                self._aos.setDEMTransform([0,0,self._focalplane_height])
                #aos.setPoseCorrectionall(len(self._prev_blob_images),[0,0,0], [0,0, self._compass_correction])
                tmp_prev_integ__blob_img = self._aos.render(self.pose_to_virtualcamera(single_poses[self._current_center_camera_id]), self._fov, img_renderering_ids)
                prev_integ__blob_img = self.divide_by_alpha(tmp_prev_integ__blob_img)
                tmp_img = np.asarray(prev_integ__blob_img[:,:,0],dtype=np.uint8)
                tmp_mask_img = np.asarray(projected_detection_mask[:,:,0],dtype=np.uint8)
                prev_tmp_img = np.zeros((tmp_img.shape[0],tmp_img.shape[1]),dtype=np.uint8)
                cv2.bitwise_and(tmp_img, tmp_mask_img, prev_tmp_img)
                gen_integral_blob_img = np.asarray(prev_tmp_img[:,:],dtype=np.uint8)
                contours,hierarchy  = cv2.findContours(np.asarray(prev_tmp_img[:,:],dtype=np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                for k in range(len(contours)):
                    contour_area = cv2.contourArea(contours[k])
                    if contour_area > prev_img_max_contour:
                        prev_img_max_contour = contour_area
            print("prev_contour_area",prev_img_max_contour)
            ##########################Check if adding image to the rendering improve integral or not########################################
            added_prev_img = True
            for i in range(len(single_images)):
                if added_prev_img:
                    print("Added View")
                    self._aos.addView(blob_images[i], single_poses[i], "DEM BlobTrack")
                else:
                    print("Replaced View")
                    self._aos.replaceView(self._aos.getViews()-1,blob_images[i], single_poses[i], "DEM BlobTrack")
                img_renderering_ids = []
                self._aos.setDEMTransform([0,0,self._focalplane_height])
                #aos.setPoseCorrectionall(self._aos.getViews(),[0,0,0], [0,0, self._compass_correction])
                tmp_tmp_integral_img = self._aos.render(self.pose_to_virtualcamera(single_poses[self._current_center_camera_id]), self._fov, img_renderering_ids)
                tmp_integral_img = self.divide_by_alpha(tmp_tmp_integral_img)
                tmp_img = np.asarray(tmp_integral_img[:,:,0],dtype=np.uint8)
                tmp_mask_img = np.asarray(projected_detection_mask[:,:,0],dtype=np.uint8)
                curr_tmp_img = np.zeros((tmp_img.shape[0],tmp_img.shape[1]),dtype=np.uint8)
                cv2.bitwise_and(tmp_img, tmp_mask_img, curr_tmp_img)
                contours,hierarchy  = cv2.findContours(np.asarray(curr_tmp_img[:,:],dtype=np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                add_img = False
                for k in range(len(contours)):
                    contour_area = cv2.contourArea(contours[k])
                    print("curr_contour_area and Image Added",prev_img_max_contour)
                    if contour_area > prev_img_max_contour:
                        prev_img_max_contour = contour_area
                        print("curr_contour_area and Image Added",prev_img_max_contour)
                        add_img = True
                if add_img:
                    print("Added to Prev List")
                    added_prev_img = True
                    self._prev_blob_images.append(blob_images[i])
                    self._prev_images.append(self._current_RGB_images_list[i])
                    if len(self._current_thermal_images_list)>= 1:
                        self._prev_thermal_images.append(self._current_thermal_images_list[i])
                    self._prev_images_poses.append(single_poses[i])
                    gen_integral_blob_img = np.asarray(curr_tmp_img[:,:],dtype=np.uint8)
                else :
                    print("Do not Add to Prev List")
                    added_prev_img = False       
        else:
            print("same as integral image")
            rgb_integral_images = self.project_images_to_all(self._aos, self._current_RGB_images_list, single_poses,self._fov,center_camera_index=None,project_images=False)
            if len(self._current_thermal_images_list)>= 1:
                thermal_integral_images = self.project_images_to_all(self._aos, self._current_thermal_images_list, single_poses,self._fov,center_camera_index=None,project_images=False)
                thermal_integral_img = np.asarray(thermal_integral_images[self._current_center_camera_id],dtype=np.uint8)
                cv2.imwrite(os.path.join( Live_Debug_Path,  str(self._integration_count)+ '_gen_thermal_integral.png'), thermal_integral_img)
                cv2.imwrite(os.path.join( Integral_Path,  str(self._integration_count)+ '_gen_thermal_integral.png'), thermal_integral_img)
            rgb_integral_img  = np.asarray(rgb_integral_images[self._current_center_camera_id],dtype=np.uint8)
            cv2.imwrite(os.path.join( Live_Debug_Path,  str(self._integration_count)+ '_gen_integral.png'), rgb_integral_img)  
            cv2.imwrite(os.path.join( Integral_Path,  str(self._integration_count)+ '_gen_integral.png'), rgb_integral_img) 

        self._aos.clearViews()
        len_info = []
        len_info.append(str(len(self._prev_images)) + '\n')
        with open(os.path.join(Live_Debug_Path,'History_Info.txt'), 'a') as f:
            f.writelines('\n'.join(len_info))
        
        print(len(self._prev_images))
        if len(self._prev_images) :
            for i in range(len(self._prev_images)):
                    self._aos.addView(self._prev_images[i], self._prev_images_poses[i], "DEM BlobTrack")
            img_renderering_ids = []
            self._aos.setDEMTransform([0,0,self._focalplane_height])
            #aos.setPoseCorrectionall(len(self._prev_blob_images),[0,0,0], [0,0, self._compass_correction])
            #tmp_rgb_integral_img = self._aos.render(self.pose_to_virtualcamera(virtual_camera_pose), self._fov, img_renderering_ids) ## edited in oct
            tmp_rgb_integral_img = self._aos.render(self.pose_to_virtualcamera(single_poses[self._current_center_camera_id]), self._fov, img_renderering_ids)
            rgb_integral_img = self.divide_by_alpha(tmp_rgb_integral_img)
            #datasets_folder_live_debug= os.path.join( r"E:\Droneswarms_Feb_clientserverdevelopment\live_debug" )
            #cv2.imwrite(os.path.join( Integral_Path,  str(self._integration_count)+ '_gen_integral.jpg'), rgb_integral_img);
            cv2.imwrite(os.path.join( Live_Debug_Path,  str(self._integration_count)+ '_gen_integral.png'), rgb_integral_img)
            cv2.imwrite(os.path.join( Integral_Path,  str(self._integration_count)+ '_gen_integral.png'), rgb_integral_img)
            if len(self._current_thermal_images_list)>= 1:
                self._aos.clearViews()
                for i in range(len(self._prev_thermal_images)):
                    self._aos.addView(self._prev_thermal_images[i], self._prev_images_poses[i], "DEM BlobTrack")
                img_renderering_ids = []
                self._aos.setDEMTransform([0,0,self._focalplane_height])
                ##tmp_thermal_integral_img = self._aos.render(self.pose_to_virtualcamera(virtual_camera_pose), self._fov, img_renderering_ids) ##edited in oct
                tmp_thermal_integral_img = self._aos.render(self.pose_to_virtualcamera(single_poses[self._current_center_camera_id]), self._fov, img_renderering_ids)
                thermal_integral_img = self.divide_by_alpha(tmp_thermal_integral_img)
                #cv2.imwrite(os.path.join( Integral_Path,  str(self._integration_count)+ '_gen_integral.jpg'), rgb_integral_img);
                cv2.imwrite(os.path.join( Live_Debug_Path,  str(self._integration_count)+ '_gen_thermal_integral.png'), thermal_integral_img)
                cv2.imwrite(os.path.join( Integral_Path,  str(self._integration_count)+ '_gen_thermal_integral.png'), thermal_integral_img)
        else :
            print("same as integral image")
            rgb_integral_images = self.project_images_to_all(self._aos, self._current_RGB_images_list, single_poses,self._fov,center_camera_index=None,project_images=False)
            if len(self._current_thermal_images_list)>= 1:
                thermal_integral_images = self.project_images_to_all(self._aos, self._current_thermal_images_list, single_poses,self._fov,center_camera_index=None,project_images=False)
                thermal_integral_img = np.asarray(thermal_integral_images[self._current_center_camera_id],dtype=np.uint8)
                cv2.imwrite(os.path.join( Live_Debug_Path,  str(self._integration_count)+ '_gen_thermal_integral.png'), thermal_integral_img)
                cv2.imwrite(os.path.join( Integral_Path,  str(self._integration_count)+ '_gen_thermal_integral.png'), thermal_integral_img)
            rgb_integral_img  = np.asarray(rgb_integral_images[self._current_center_camera_id],dtype=np.uint8)
            cv2.imwrite(os.path.join( Live_Debug_Path,  str(self._integration_count)+ '_gen_integral.png'), rgb_integral_img) 
            cv2.imwrite(os.path.join( Integral_Path,  str(self._integration_count)+ '_gen_integral.png'), rgb_integral_img) 
        self._integration_count = self._integration_count + 1
        return rgb_integral_img, prev_img_max_contour,gen_integral_blob_img

        
class drone_pso:
    _fov = 45#22.815436217896945#43.10803984095769#43.50668199945787#50.815436217896945
    _rx_threshold = 0.999
    _no_of_drones = 10
    _distance_factor = 2.0
    _scanning_bearing = 0.0
    _empty_scene_rx_blob_size_threshold = 500.0
    _scandirec_waypoint_distance = 3.0
    _init = True
    _aos = None
    _images = None
    _poses = None
    _previous_drone_pos = None
    _center_waypoint = None
    _changing_back_speed = 2.0
    _cognitive_local_fac = 0.2
    _social_global_fac = 0.2
    _minimum_drone_distance = _distance_factor
    _metric_cal = None
    _debug = False
    _current_gps_waypoints = None
    _waypoint_count = 0
    
    def __init__(self,aos_ptr,no_drones,fieldofview,rxthreshold,distancebtwndrones,scanningdirection,scanning_direction_waypoint_distance,emptysceneblobthreshold, starting_loc_gps, changingtolinearspeed, localfac, globalfac, minimumdistancebetweendrone, debug_mode = False):
        
        self._aos = aos_ptr
        self._no_of_drones = no_drones
        self._fov = fieldofview
        self._rx_threshold = rxthreshold
        self._distance_factor = distancebtwndrones
        self._scanning_bearing = scanningdirection
        self._empty_scene_rx_blob_size_threshold = emptysceneblobthreshold
        self._center_waypoint = starting_loc_gps
        self._changing_back_speed = changingtolinearspeed
        self._cognitive_local_fac = localfac
        self._social_global_fac = globalfac
        self._minimum_drone_distance = minimumdistancebetweendrone
        self._metric_cal = evaluate_metric(self._aos, self._fov,self._rx_threshold, dem_height= 0.0,compasscorrection=0.0,debug_mode=debug_mode)
        self._debug = debug_mode
        self._scandirec_waypoint_distance = scanning_direction_waypoint_distance
        self._waypoint_count = 0
        self._continous_converging = 0
        self._startingcentreposition = [0,-12]  #[0,-12]
        self._blob_person_position = [0,10]
        self._current_blob_position = [0,10]

        
        if self._debug:
            np.random.seed(100)
        self._centered_drone_array = np.arange(-(int)((self._no_of_drones-1)/2),(int)((self._no_of_drones+2)/2),1) * self._distance_factor
        if self._debug:
            self._direction_vector = [3.0,3.0]
        else :
            self._direction_vector = [np.sin(np.deg2rad( self._scanning_bearing)), np.cos(np.deg2rad( self._scanning_bearing))]
        self._center_east,self._center_north,self._zone_number, self._zone_letter = utm.from_latlon(self._center_waypoint[0],self._center_waypoint[1])
        
        self._vec_perpend_to_scan_dir = self.perpendicular(self.normalize(self._direction_vector))
        print(self._vec_perpend_to_scan_dir)
        print(self._direction_vector)
        
        self._rel_drone_pos = np.transpose(np.array(list(tuple(self._vec_perpend_to_scan_dir* drone_pos_to_center) for drone_pos_to_center in self._centered_drone_array)))
        print(self._rel_drone_pos.shape)
       # self._rel_drone_pos = self._rel_drone_pos + (np.transpose(np.tile(np.array(self._startingcentreposition),(self._no_of_drones,1))))
        
        print(self._centered_drone_array)
        print(self._rel_drone_pos.shape)
        print(self._rel_drone_pos)
        
        
        self._mean_drone_pos = np.mean(self._rel_drone_pos,1).reshape(2,1)
        self._rel_drone_pos = self._rel_drone_pos - np.repeat(self._mean_drone_pos,self._no_of_drones,axis = 1)
        #self._rel_drone_pos = self._rel_drone_pos + (np.transpose(np.tile(np.array(self._startingcentreposition),(self._no_of_drones,1))))

        self._rand_velocity_vec = self._scanning_dir_vec = np.random.randn(2, self._no_of_drones) * 0.1
        normalized_scan_vec = self.normalize(self._direction_vector)
        
        for i in range(self._no_of_drones):
            #self._scanning_dir_vec[:,i] = (self._direction_vector[0],self._direction_vector[1])
            self._scanning_dir_vec[:,i] = (normalized_scan_vec[0]*self._scandirec_waypoint_distance,normalized_scan_vec[1]*self._scandirec_waypoint_distance)
        #self._scanning_dir_vec = np.repeat(self._direction_vector,self._no_of_drones,axis = 1)

        drones_next_east_loc = self._rel_drone_pos[0,:] + self._center_east
        drones_next_north_loc = self._rel_drone_pos[1,:] + self._center_north
       

        drones_next_lat,drones_next_lon = utm.to_latlon(drones_next_east_loc, drones_next_north_loc, self._zone_number, self._zone_letter)
     
        drones_next_loc = list(zip(drones_next_lat, drones_next_lon))
        
        self._previous_drone_pos = self._rel_drone_pos + (np.transpose(np.tile(np.array(self._startingcentreposition),(self._no_of_drones,1))))  #self._rel_drone_pos
        self._current_gps_waypoints = drones_next_loc
        
        #return drones_next_loc, self._previous_drone_pos
    def set_scanning_direction(self, scanning_bearing):
        self._scanning_bearing = scanning_bearing
        if self._debug:
            self._direction_vector = [3.0,3.0]
        else :
            self._direction_vector = [np.sin(np.deg2rad( self._scanning_bearing)), np.cos(np.deg2rad( self._scanning_bearing))]
  
        self._vec_perpend_to_scan_dir = self.perpendicular(self.normalize(self._direction_vector))
        print(self._vec_perpend_to_scan_dir)
        print(self._direction_vector)
        
        self._rel_drone_pos = np.transpose(np.array(list(tuple(self._vec_perpend_to_scan_dir* drone_pos_to_center) for drone_pos_to_center in self._centered_drone_array)))
        print(self._rel_drone_pos.shape)
        # self._rel_drone_pos = self._rel_drone_pos + (np.transpose(np.tile(np.array(self._startingcentreposition),(self._no_of_drones,1))))
        
        print(self._centered_drone_array)
        print(self._rel_drone_pos.shape)
        print(self._rel_drone_pos)
    
        self._mean_drone_pos = np.mean(self._rel_drone_pos,1).reshape(2,1)
        self._rel_drone_pos = self._rel_drone_pos - np.repeat(self._mean_drone_pos,self._no_of_drones,axis = 1)
        #self._rel_drone_pos = self._rel_drone_pos + (np.transpose(np.tile(np.array(self._startingcentreposition),(self._no_of_drones,1))))

        normalized_scan_vec = self.normalize(self._direction_vector)
        for i in range(self._no_of_drones):
            self._scanning_dir_vec[:,i] = (normalized_scan_vec[0]*self._scandirec_waypoint_distance,normalized_scan_vec[1]*self._scandirec_waypoint_distance)

    def set_scan_direction_distance(self,scanning_direction_waypoint_distance):
        self._scandirec_waypoint_distance = scanning_direction_waypoint_distance
        normalized_scan_vec = self.normalize(self._direction_vector)
        for i in range(self._no_of_drones):
            self._scanning_dir_vec[:,i] = (normalized_scan_vec[0]*self._scandirec_waypoint_distance,normalized_scan_vec[1]*self._scandirec_waypoint_distance)

    def set_emptyblob_threshold(self, contour_area_threshold):
        self._empty_scene_rx_blob_size_threshold = contour_area_threshold

    def get_rx_threshold(self):
        return self._empty_scene_rx_blob_size_threshold
    
    def determine_emptyblob_threshold(self,single_images,site_poses, fov = None, rxthreshold =  None, dem_height = None, compasscorrection = None , pos = None):
        center_rgb_integral_image, leader_rx_image, Leader_index, leader_contour = self._metric_cal.get_relevent_info(single_images,site_poses, fov, rxthreshold, dem_height, compasscorrection, pos)
        return center_rgb_integral_image, leader_rx_image, Leader_index, leader_contour
        
    def get_current_waypoints(self):
        return self._current_gps_waypoints, self._previous_drone_pos

    def get_leader_info(self, images, poses):
        position = self._previous_drone_pos
        leader_index, leader_contour_area, mask_img, camera_id,leader_blob_image, leader_rx_image,rect = self._metric_cal.find_leader_within_images(images, poses, self._fov, rxthreshold=None, pos = position)

    def get_waypoints_with_pso(self,images, poses, virtual_camera_pose):
        
        self._waypoint_count = self._waypoint_count + 1
        position = self._previous_drone_pos
        #if self._debug:
            #print("Previous Pos for Finding Leader", self._previous_drone_pos)
        leader_index, leader_contour_area, mask_img, camera_id,leader_blob_image, leader_rx_image,rect = self._metric_cal.find_leader_within_images(images, poses, self._fov, rxthreshold=None, pos = position)
        if (leader_contour_area < self._empty_scene_rx_blob_size_threshold):
            #print("Project to Old Mask")
            integral_image, prev_contour_area, gen_integral_blob_img = self._metric_cal.integrate_informative_images(images, poses, virtual_camera_pose, None, None)
            #prev_contour_area = leader_contour_area
        else :
            #print("Project to New Mask")
            integral_image, prev_contour_area, gen_integral_blob_img = self._metric_cal.integrate_informative_images(images, poses, virtual_camera_pose, mask_img, poses[camera_id])
        info = []
        info.append('WayPoint_Count ' + str(self._waypoint_count) + ' \n')
        info.append('Leader Index ' + str(leader_index) + ' \n')
        info.append('Leader Contour ' + str(leader_contour_area) + ' \n')
        info.append('Contour Threshold ' + str(self._empty_scene_rx_blob_size_threshold) + ' \n')
        
        #integral_image, integral_camera_pose = self._metric_cal.integrate_informative_images(self._aos, images, poses, self._prev_images, self._prev_poses, virtual_camera_pose )
        #if self._debug:
            #print("Leader Index, Leader Area", leader_index, leader_contour_area)
        if (leader_contour_area < self._empty_scene_rx_blob_size_threshold): #or (iteration >=exit_iteration):
            self._continous_converging = 0
            curr_drone_rel_pos = np.repeat(np.mean(self._previous_drone_pos,1).reshape(2,1),self._no_of_drones,axis = 1) + self._rel_drone_pos #line formation around center of gravety of previous swarm position
            info.append('Previous Drone Positions ' + str(self._previous_drone_pos[0,:]) + '_' +  str(self._previous_drone_pos[1,:])+ ' \n')
            info.append('Mean Previous Drone Positions ' + str(np.repeat(np.mean(self._previous_drone_pos,1).reshape(2,1),self._no_of_drones,axis = 1)[0,:]) + '_' +  str(np.repeat(np.mean(self._previous_drone_pos,1).reshape(2,1),self._no_of_drones,axis = 1)[1,:])+ ' \n')
            info.append('relative Drone Positions ' + str(self._rel_drone_pos[0,:]) + '_' +  str(self._rel_drone_pos[1,:])+ ' \n')
            info.append('current starting Drone Positions ' + str(curr_drone_rel_pos[0,:]) + '_' +  str(curr_drone_rel_pos[1,:])+ ' \n')
            #velocity_vec = self._scanning_dir_vec + self._changing_back_speed * (curr_drone_rel_pos - self._previous_drone_pos)
            #velocity_vec = self._scanning_dir_vec + self._changing_back_speed * self.normalize_vectors(curr_drone_rel_pos - self._previous_drone_pos)
            velocity_vec = self._scanning_dir_vec + self._changing_back_speed * (curr_drone_rel_pos - self._previous_drone_pos)

            info.append('scanning direction vector ' + str(self._scanning_dir_vec[0,:]) + '_' +  str(self._scanning_dir_vec[1,:])+ ' \n')
            info.append('changing back speed ' + str(self._changing_back_speed) + ' \n')
            info.append('velocity vector ' + str(velocity_vec[0,:]) + '_' +  str(velocity_vec[1,:])+ ' \n')
            #info.append('Normalize velocity vector ' + str(velocity_vec_norm[0,:]) + '_' +  str(velocity_vec_norm[1,:])+ ' \n')
            new_drone_rel_pos = self._previous_drone_pos + velocity_vec
            self._previous_drone_pos = new_drone_rel_pos
            info.append('Moving Straight ' + str(new_drone_rel_pos[0,:]) + '_' +  str(new_drone_rel_pos[1,:])+ ' \n')
        else :
            self._continous_converging = self._continous_converging + 1
            if self._debug:
                np.random.seed(100)
            rand_mov_vec = np.random.rand(2, self._no_of_drones)*2 - np.ones(2*self._no_of_drones).reshape(2, self._no_of_drones)
            #if self._debug:
                #print("rand_mov_vec", rand_mov_vec)
                #print("leader pos", self._previous_drone_pos[:,leader_index].reshape(-1,1))
            #resp_vel_vec = self._cognitive_local_fac * self.normalize(rand_mov_vec) + self._social_global_fac * self.normalize((self._previous_drone_pos[:,leader_index].reshape(-1,1) - self._previous_drone_pos))
            resp_vel_vec = self._cognitive_local_fac * self.normalize_vectors(rand_mov_vec) + self._social_global_fac * self.normalize_vectors((self._previous_drone_pos[:,leader_index].reshape(-1,1) - self._previous_drone_pos))
            #if self._debug:
                #print("Previous Pos", self._previous_drone_pos)
                #print("Velocity Vector", resp_vel_vec) 
            new_drone_rel_pos = self._previous_drone_pos + resp_vel_vec
            info.append('Converging Before Rutherford Scattering ' + str(new_drone_rel_pos[0,:]) + '_' +  str(new_drone_rel_pos[1,:])+ ' \n')
            #if self._debug:
                #print("new drone_pos", new_drone_rel_pos) 
            new_drone_rel_pos = self.rutherford_scattering(new_drone_rel_pos,self._no_of_drones,self._minimum_drone_distance) # minimal distance constraint (Rutherford Scattering)
            #if self._debug:
                #print("ruttherfordScanning", new_drone_rel_pos) 
            self._previous_drone_pos = new_drone_rel_pos
            info.append('Minimum Distance Within Rutherford Scattering ' + str(self._minimum_drone_distance) + ' \n')
            info.append('Converging After Rutherford Scattering ' + str(new_drone_rel_pos[0,:]) + '_' + str(new_drone_rel_pos[1,:])+ ' \n')
            

        drones_next_east_loc = new_drone_rel_pos[0,:] + self._center_east
        drones_next_north_loc = new_drone_rel_pos[1,:] + self._center_north

        drones_next_lat,drones_next_lon = utm.to_latlon(drones_next_east_loc,drones_next_north_loc, self._zone_number, self._zone_letter)
        info.append('drones next lat ' + str(drones_next_lat) + ' drones next lon ' + str(drones_next_lon)+ ' \n')
        drones_next_loc = zip(drones_next_lat, drones_next_lon)
        #print(list(drones_next_loc))
        self._current_gps_waypoints = drones_next_loc
        #datasets_folder_live_debug= os.path.join( r"E:\Droneswarms_Feb_clientserverdevelopment\live_debug" )
        with open(os.path.join(Live_Debug_Path,'WayPoint_Info.txt'), 'a') as f:
            f.writelines('\n'.join(info))
        return drones_next_loc, self._previous_drone_pos, integral_image,leader_contour_area, leader_index, prev_contour_area,camera_id,leader_blob_image, leader_rx_image,gen_integral_blob_img,rect
    
    def get_waypoints_with_pso_motion(self,images, poses, virtual_camera_pose):
        if self._waypoint_count == 0:
            self._previous_drone_pos  = np.array([[9.9717100350285, 7.509278207141478, 7.9509800794951735, 10.435511445470253, -0.7602028575178091, 13.721811741972076, 1.924510020547081, 4.666728601049116, 3.231715985565085, 0.936382834234614], [-8.656215061944506, -5.710321502200583, -0.16034449090046393, 3.5812591450458786, 0.30581173782537846, -5.231449516231961, -7.244081751496854, -2.52686032502034, 3.1534761204635524, -3.2818597748696416]], dtype = 'float')
            ###below is for old drones n=10 300trees/ha without altitude bias
            #self._previous_drone_pos  =  np.array([[1.8115134503385846, 10.133639847964734, 12.629432704779868, 11.903718818156282, 6.7633418126185845, 4.691795821233404, 5.602373545591946, 8.155933054264636, 7.186430847096473, 0.3061385423385319], [-5.02637415149285, 5.097264629755322, 1.958122637084436, -3.986459164351714, 7.253806446287938, 1.848477920955482, -6.393277670197103, 1.9794189449672637, -2.746389521970734, 1.4854112721404695]], dtype = 'float')

        self._waypoint_count = self._waypoint_count + 1
        position = self._previous_drone_pos
        personorientation = 100
        #if self._debug:
            #print("Previous Pos for Finding Leader", self._previous_drone_pos)
        leader_index, leader_contour_area, mask_img, camera_id,leader_blob_image, leader_rx_image,rect = self._metric_cal.find_leader_within_images(images, poses, self._fov, rxthreshold=None, pos = position)
        # if (leader_contour_area < self._empty_scene_rx_blob_size_threshold):
        #     self._continous_converging = 0
        #     #print("Project to Old Mask")
        #     #integral_image, prev_contour_area, gen_integral_blob_img = self._metric_cal.integrate_informative_images(images, poses, virtual_camera_pose, None, None)
        #     integral_image, prev_contour_area, gen_integral_blob_img = self._metric_cal.integrate_informative_images(images, poses, virtual_camera_pose, mask_img, poses[camera_id])
        #     motion = []
        #     motion.append('Waypoint ' + str(self._waypoint_count) + ' \n')
        #     motion.append('No convergence'  + ' \n') 
        #     # motion.append('person ' + str(person) + ' \n')
        #     motion.append('No convergence'  + ' \n')  
        #     motion.append('Scanning_direction ' + str(self._scanning_dir_vec) + ' \n')
        #     #motion.append('personorientation ' + str(personorientation) + ' \n')
            
        #     #motion.append('Distancetravelledbyperson ' + str(person_distance) + ' \n')
        #     motion.append('Scanning_direction_Waypoint_Distance_c3 ' + str(self._scandirec_waypoint_distance) + ' \n')
        #     positiondummy = [60,60]  # dummy number
        #     rectdummy = [59,59,2,2]
        #     import json
        #     bd ={
        #             self._waypoint_count:positiondummy,
                    
        #         }



        #     with open(r'C:\Users\Rakesh\Downloads\blobpositions.json','r+') as file4:
            
        #         data4 = json.load(file4)
        #         data4.update(bd)
        #         file4.seek(0)
        #         json.dump(data4, file4)  
                
                
             
        #     rt9 ={
        #             self._waypoint_count:rectdummy,
                    
        #         }



        #     with open(r'C:\Users\Rakesh\Downloads\rectpositions.json','r+') as file10:
            
        #             data10 = json.load(file10)
        #             data10.update(rt9)
        #             file10.seek(0)
        #             json.dump(data10, file10)    
        #     #prev_contour_area = leader_contour_area
        # else :
        #     self._continous_converging = self._continous_converging + 1
        #     #print("Project to New Mask")
        #     integral_image, prev_contour_area, gen_integral_blob_img = self._metric_cal.integrate_informative_images(images, poses, virtual_camera_pose, mask_img, poses[camera_id])
        #     previous_blob_personposition = self._blob_person_position 
            
            
            
            
            
            
        #     altitude_list = [43,41,39,37,35,36,38,40,42,44]
        #     new_leader_index = leader_index
        #     coverage = 2*altitude_list[new_leader_index]*np.tan(np.deg2rad(25))
        #     resolution = 512
        #     metretopixels = coverage/resolution
        #     blob_posx = rect[0] + rect[2]/2
        #     blob_posy = rect[1] + rect[3]/2
        #     blob_posxinpixels = blob_posx * metretopixels
        #     blob_posyinpixels = blob_posy * metretopixels
        #     blobposition = [blob_posxinpixels,blob_posyinpixels]
        #     ref_loc = self._previous_drone_pos
        #     integralposition_centre = [ref_loc[0][new_leader_index], ref_loc[1][new_leader_index]]
        #     integral_startposition = [integralposition_centre[0] - coverage/2, integralposition_centre[1] - coverage/2]
        #     blob_personposition = [blobposition[0] +(integral_startposition[0]), blobposition[1] + (integral_startposition[1])]
            

        #     dronemeanx=(np.mean(ref_loc[0]))
        #     dronemeany=(np.mean(ref_loc[1]))
        #     dronemeanpos = [dronemeanx,dronemeany]
        #     dy = (-blob_personposition[1]-(-dronemeany))
        #     dx = (blob_personposition[0]-dronemeanx)

        #     Scanning_direction = np.arctan2(dy,dx)
        #     Scanning_direction = ((np.degrees(Scanning_direction))%360)+90
        #     print(Scanning_direction)
        #     if Scanning_direction >=360 :
        #         Scanning_direction =  Scanning_direction - 360                             #need to switch sign of yaxis positive is down and negative is up
        #         print(Scanning_direction)


            
        #     # dyp = (-blob_personposition[1]-(-previous_blob_personposition[1]))
        #     # dxp = (blob_personposition[0]-previous_blob_personposition[0])
        #     # personorientation = np.arctan2(dyp,dxp)
        #     # personorientation = ((np.degrees(personorientation))%360)+90
        #     # print(personorientation)
        #     # if personorientation >=360 :
        #     #     personorientation = personorientation - 360                             #need to switch sign of yaxis positive is down and negative is up
        #     #     print(personorientation)
            
            
            
            
        #     c = []
        #     drone_speed = 10
        #     for i in range (0, 10):
        #         distancedrone = math.sqrt( ((ref_loc[0][i]-prevwaypoints[0][i])**2)+((ref_loc[1][i]-prevwaypoints[1][i])**2) )
        #         c.append(distancedrone)
                
        #     print(c)
        #     new_max_distance =  max(c)    
        #     print(max(c))
            
        #     drone_totaltime = (new_max_distance/drone_speed)
            
            
            
        #     delta = 0.3
        #     person_distance = math.sqrt( (blob_personposition[0] - previous_blob_personposition[0] )**2 + (blob_personposition[1] - previous_blob_personposition[1])**2 )
            
        #     # person_time = drone_totaltime
        #     # person_calculatedspeed = person_distance/person_time
        #     # person_simspeed = 3    
        #     c3 = person_distance #person_calculatedspeed *  person_time # to be updated
        #     Scanning_direction_Waypoint_Distance = c3 + delta #c3   
        #     if self._continous_converging > 1:
        #         self.set_scan_direction_distance(Scanning_direction_Waypoint_Distance)
            
        #     import json
        #     bd ={
        #             self._waypoint_count:blob_personposition,
                    
        #         }

            

        #     with open(r'C:\Users\Rakesh\Downloads\blobpositions.json','r+') as file3:
            
        #             data3 = json.load(file3)
        #             data3.update(bd)
        #             file3.seek(0)
        #             json.dump(data3, file3)  
                    
                    
        #     import json
        #     rt ={
        #             self._waypoint_count:rect,
                    
        #         }



        #     with open(r'C:\Users\Rakesh\Downloads\rectpositions.json','r+') as file6:
            
        #             data6 = json.load(file6)
        #             data6.update(rt)
        #             file6.seek(0)
        #             json.dump(data6, file6)   
                    
                    
        #     import json
        #     sd1 ={
        #             self._waypoint_count:Scanning_direction_Waypoint_Distance,
                    
        #         }



        #     with open(r'C:\Users\Rakesh\Downloads\scanningwaypointdistance.json','r+') as file11:
            
        #             data11 = json.load(file11)
        #             data11.update(sd1)
        #             file11.seek(0)
        #             json.dump(data11, file11)                   
        

        #     # print("simulated_personposition",person)
        #     print("previous_blob_personposition",previous_blob_personposition)
        #     print("coverage",coverage)
        #     print("new_leader_index",new_leader_index)
        #     print("blobposition",blobposition)
        #     print("integralposition_centre",integralposition_centre)
        #     print("integral_startposition",integral_startposition)
        #     print("currentblob_personposition",blob_personposition)
        #     print("dronemeanpos",dronemeanpos)
        #     print("Scanning_direction",Scanning_direction)
        #     #print("personorientation",personorientation)
        #     # print("timeforcurrentiteration",person_time)
        #     print("Distancetravelledbyperson",person_distance)
        #     print("c3",Scanning_direction_Waypoint_Distance)
            
            
        #     motion = []
        #     motion.append('Waypoint ' + str(self._waypoint_count) + ' \n')
        #     motion.append('rect ' + str(rect) + ' \n')
        #     motion.append('previous_blob_personposition ' + str(previous_blob_personposition) + ' \n')
        #     motion.append('coverage ' + str(coverage) + ' \n')
        #     motion.append('new_leader_index ' + str(new_leader_index) + ' \n')
        #     motion.append('blobposition ' + str(blobposition) + ' \n')
        #     motion.append('integralposition_centre ' + str(integralposition_centre) + ' \n')
        #     motion.append('integral_startposition ' + str(integral_startposition) + ' \n')
        #     motion.append('currentblob_personposition ' + str(blob_personposition) + ' \n')
        #     motion.append('dronemeanpos ' + str(dronemeanpos) + ' \n')
        #     motion.append('Scanning_direction ' + str(Scanning_direction) + ' \n')
        #     #motion.append('personorientation ' + str(personorientation) + ' \n')
        #     # motion.append('timeforcurrentiteration ' + str(person_time) + ' \n')
        #     motion.append('Distancetravelledbyperson ' + str(person_distance) + ' \n')
        #     motion.append('Scanning_direction_Waypoint_Distance_c3 ' + str(Scanning_direction_Waypoint_Distance) + ' \n')
        #     self.set_scanning_direction(Scanning_direction)
            
        #     self._blob_person_position = blob_personposition

        # with open(os.path.join(Live_Debug_Path,'motion.txt'), 'a') as f:
        #     f.writelines('\n'.join(motion))            
        info = []
        info.append('WayPoint_Count ' + str(self._waypoint_count) + ' \n')
        info.append('Leader Index ' + str(leader_index) + ' \n')
        info.append('Leader Contour ' + str(leader_contour_area) + ' \n')
        info.append('Contour Threshold ' + str(self._empty_scene_rx_blob_size_threshold) + ' \n')
        
        #integral_image, integral_camera_pose = self._metric_cal.integrate_informative_images(self._aos, images, poses, self._prev_images, self._prev_poses, virtual_camera_pose )
        #if self._debug:
            #print("Leader Index, Leader Area", leader_index, leader_contour_area)
        if (leader_contour_area < self._empty_scene_rx_blob_size_threshold): #or (iteration >=exit_iteration):

            # dronemeanx=(np.mean(self._previous_drone_pos[0]))
            # dronemeany=(np.mean(self._previous_drone_pos[1]))
            # dronemeanpos = [dronemeanx,dronemeany]
            # dy = (-self._blob_person_position[1]-(-dronemeany))
            # dx = (self._blob_person_position[0]-dronemeanx)

            # Scanning_direction = np.arctan2(dy,dx)
            # Scanning_direction = ((np.degrees(Scanning_direction))%360)+90
            # print(Scanning_direction)
            # if Scanning_direction >=360 :
            #     Scanning_direction =  Scanning_direction - 360                             #need to switch sign of yaxis positive is down and negative is up
            #     print(Scanning_direction)

            # self.set_scanning_direction(Scanning_direction)
            curr_drone_rel_pos = np.repeat(np.mean(self._previous_drone_pos,1).reshape(2,1),self._no_of_drones,axis = 1) + self._rel_drone_pos #line formation around center of gravety of previous swarm position
            info.append('Previous Drone Positions ' + str(self._previous_drone_pos[0,:]) + '_' +  str(self._previous_drone_pos[1,:])+ ' \n')
            info.append('Mean Previous Drone Positions ' + str(np.repeat(np.mean(self._previous_drone_pos,1).reshape(2,1),self._no_of_drones,axis = 1)[0,:]) + '_' +  str(np.repeat(np.mean(self._previous_drone_pos,1).reshape(2,1),self._no_of_drones,axis = 1)[1,:])+ ' \n')
            info.append('relative Drone Positions ' + str(self._rel_drone_pos[0,:]) + '_' +  str(self._rel_drone_pos[1,:])+ ' \n')
            info.append('current starting Drone Positions ' + str(curr_drone_rel_pos[0,:]) + '_' +  str(curr_drone_rel_pos[1,:])+ ' \n')
            info.append('Mean Current Drone Positions ' + str(np.repeat(np.mean(curr_drone_rel_pos,1).reshape(2,1),self._no_of_drones,axis = 1)[0,:]) + '_' +  str(np.repeat(np.mean(curr_drone_rel_pos,1).reshape(2,1),self._no_of_drones,axis = 1)[1,:])+ ' \n')

            #velocity_vec = self._scanning_dir_vec + self._changing_back_speed * (curr_drone_rel_pos - self._previous_drone_pos)
            #velocity_vec = self._scanning_dir_vec + self._changing_back_speed * self.normalize_vectors(curr_drone_rel_pos - self._previous_drone_pos)
            velocity_vec = self._scanning_dir_vec + self._changing_back_speed * (curr_drone_rel_pos - self._previous_drone_pos)

            info.append('scanning direction vector ' + str(self._scanning_dir_vec[0,:]) + '_' +  str(self._scanning_dir_vec[1,:])+ ' \n')
            info.append('changing back speed ' + str(self._changing_back_speed) + ' \n')
            info.append('velocity vector ' + str(velocity_vec[0,:]) + '_' +  str(velocity_vec[1,:])+ ' \n')
            #info.append('Normalize velocity vector ' + str(velocity_vec_norm[0,:]) + '_' +  str(velocity_vec_norm[1,:])+ ' \n')
            new_drone_rel_pos = self._previous_drone_pos + velocity_vec
            ###############xxxxxxxxxxxxxxxCurrent iteration ref loc = new_drone_rel_pos and previous loc = self._previous_drone_posxxxxxxxxxxxxxxxxxxxxxxxx
            self._previous_drone_pos = new_drone_rel_pos
            info.append('Moving Straight ' + str(new_drone_rel_pos[0,:]) + '_' +  str(new_drone_rel_pos[1,:])+ ' \n')
            
           
            self._continous_converging = 0
            #print("Project to Old Mask")
            #integral_image, prev_contour_area, gen_integral_blob_img = self._metric_cal.integrate_informative_images(images, poses, virtual_camera_pose, None, None)
            integral_image, prev_contour_area, gen_integral_blob_img = self._metric_cal.integrate_informative_images(images, poses, virtual_camera_pose, mask_img, poses[camera_id])
            motion = []
            motion.append('Waypoint ' + str(self._waypoint_count) + ' \n')
            motion.append('No convergence'  + ' \n') 
            # motion.append('person ' + str(person) + ' \n')
            motion.append('No convergence'  + ' \n')  
            #motion.append('Scanning_direction ' + str(Scanning_direction) + ' \n')
            #motion.append('personorientation ' + str(personorientation) + ' \n')
            
            #motion.append('Distancetravelledbyperson ' + str(person_distance) + ' \n')
            motion.append('Scanning_direction_Waypoint_Distance_c3 ' + str(self._scandirec_waypoint_distance) + ' \n')
            positiondummy = [60,60]  # dummy number
            rectdummy = [59,59,2,2]
            import json
            bd ={
                    self._waypoint_count:positiondummy,
                    
                }



            with open(r'C:\Users\Rakesh\Downloads\blobpositions.json','r+') as file4:
            
                data4 = json.load(file4)
                data4.update(bd)
                file4.seek(0)
                json.dump(data4, file4)  
                
                
             
            rt9 ={
                    self._waypoint_count:rectdummy,
                    
                }



            with open(r'C:\Users\Rakesh\Downloads\rectpositions.json','r+') as file10:
            
                    data10 = json.load(file10)
                    data10.update(rt9)
                    file10.seek(0)
                    json.dump(data10, file10)  
                    
                    
            sd14 ={
                    self._waypoint_count:self._scandirec_waypoint_distance,
                    
                }



            with open(r'C:\Users\Rakesh\Downloads\scanningwaypointdistance.json','r+') as file14:
            
                    data14 = json.load(file14)
                    data14.update(sd14)
                    file14.seek(0)
                    json.dump(data14, file14)         
                     
            #prev_contour_area = leader_contour_area
        else :
            if self._debug:
                np.random.seed(100)
            rand_mov_vec = np.random.rand(2, self._no_of_drones)*2 - np.ones(2*self._no_of_drones).reshape(2, self._no_of_drones)
            #if self._debug:
                #print("rand_mov_vec", rand_mov_vec)
                #print("leader pos", self._previous_drone_pos[:,leader_index].reshape(-1,1))
            #resp_vel_vec = self._cognitive_local_fac * self.normalize(rand_mov_vec) + self._social_global_fac * self.normalize((self._previous_drone_pos[:,leader_index].reshape(-1,1) - self._previous_drone_pos))
            resp_vel_vec = self._cognitive_local_fac * self.normalize_vectors(rand_mov_vec) + self._social_global_fac * self.normalize_vectors((self._previous_drone_pos[:,leader_index].reshape(-1,1) - self._previous_drone_pos))
            #if self._debug:
                #print("Previous Pos", self._previous_drone_pos)
                #print("Velocity Vector", resp_vel_vec) 
            new_drone_rel_pos = self._previous_drone_pos + resp_vel_vec
            info.append('Converging Before Rutherford Scattering ' + str(new_drone_rel_pos[0,:]) + '_' +  str(new_drone_rel_pos[1,:])+ ' \n')
            #if self._debug:
                #print("new drone_pos", new_drone_rel_pos) 
            new_drone_rel_pos = self.rutherford_scattering(new_drone_rel_pos,self._no_of_drones,self._minimum_drone_distance) # minimal distance constraint (Rutherford Scattering)
            #if self._debug:
                #print("ruttherfordScanning", new_drone_rel_pos) 
            ###############xxxxxxxxxxxxxxxCurrent iteration ref loc = new_drone_rel_pos and previous loc = self._previous_drone_posxxxxxxxxxxxxxxxxxxxxxxxx
            
            currentdronepos = new_drone_rel_pos
            previousdronepos = self._previous_drone_pos
            
            self._previous_drone_pos = new_drone_rel_pos
            info.append('Minimum Distance Within Rutherford Scattering ' + str(self._minimum_drone_distance) + ' \n')
            info.append('Converging After Rutherford Scattering ' + str(new_drone_rel_pos[0,:]) + '_' + str(new_drone_rel_pos[1,:])+ ' \n')
            
            
            self._continous_converging = self._continous_converging + 1
            #print("Project to New Mask")
            integral_image, prev_contour_area, gen_integral_blob_img = self._metric_cal.integrate_informative_images(images, poses, virtual_camera_pose, mask_img, poses[camera_id])
            previous_blob_personposition = self._blob_person_position 
            
            
            
            
            
            
            altitude_list = [43,41,39,37,35,36,38,40,42,44] #[35,36,37,38,39,40,41,42,43,44]  #[40,40,40,40,40,40,40,40,40,40] #[43,41,39,37,35,36,38,40,42,44]
            new_leader_index = leader_index
            coverage = 2*altitude_list[new_leader_index]*np.tan(np.deg2rad(25))
            resolution = 512
            metretopixels = coverage/resolution
            blob_posx = rect[0] + rect[2]/2
            blob_posy = rect[1] + rect[3]/2
            blob_posxinpixels = blob_posx * metretopixels
            blob_posyinpixels = blob_posy * metretopixels
            blobposition = [blob_posxinpixels,blob_posyinpixels]
            ref_loc = previousdronepos
            integralposition_centre = [ref_loc[0][new_leader_index], ref_loc[1][new_leader_index]]
            integral_startposition = [integralposition_centre[0] - coverage/2, integralposition_centre[1] - coverage/2]
            blob_personposition = [blobposition[0] +(integral_startposition[0]), blobposition[1] + (integral_startposition[1])]
            

            dronemeanx=(np.mean(ref_loc[0]))
            dronemeany=(np.mean(ref_loc[1]))
            dronemeanpos = [dronemeanx,dronemeany]
            dy = (-blob_personposition[1]-(-dronemeany))
            dx = (blob_personposition[0]-dronemeanx)

            Scanning_direction = np.arctan2(dy,dx)
            Scanning_direction = ((np.degrees(Scanning_direction))%360)+90
            print(Scanning_direction)
            if Scanning_direction >=360 :
                Scanning_direction =  Scanning_direction - 360                             #need to switch sign of yaxis positive is down and negative is up
                print(Scanning_direction)


            
            # dyp = (-blob_personposition[1]-(-previous_blob_personposition[1]))
            # dxp = (blob_personposition[0]-previous_blob_personposition[0])
            # personorientation = np.arctan2(dyp,dxp)
            # personorientation = ((np.degrees(personorientation))%360)+90
            # print(personorientation)
            # if personorientation >=360 :
            #     personorientation = personorientation - 360                             #need to switch sign of yaxis positive is down and negative is up
            #     print(personorientation)
            
            
            
            
            c = []
            drone_speed = 10
            for i in range (0, 10):
                distancedrone = math.sqrt( ((currentdronepos[0][i]-previousdronepos[0][i])**2)+((currentdronepos[1][i]-previousdronepos[1][i])**2) )
                c.append(distancedrone)
                
            print(c)
            new_max_distance =  max(c)    
            print(max(c))
            
            drone_totaltime = (new_max_distance/drone_speed)
            
            
            
            delta = 1 * drone_totaltime
            person_distance = math.sqrt( (blob_personposition[0] - previous_blob_personposition[0] )**2 + (blob_personposition[1] - previous_blob_personposition[1])**2 )
            
            # person_time = drone_totaltime
            # person_calculatedspeed = person_distance/person_time
            # person_simspeed = 3    
            c3 = person_distance #person_calculatedspeed *  person_time # to be updated
            Scanning_direction_Waypoint_Distance = c3 + delta #c3   
            if self._continous_converging > 1:
                self.set_scan_direction_distance(Scanning_direction_Waypoint_Distance)
            
            import json
            bd ={
                    self._waypoint_count:blob_personposition,
                    
                }

            

            with open(r'C:\Users\Rakesh\Downloads\blobpositions.json','r+') as file3:
            
                    data3 = json.load(file3)
                    data3.update(bd)
                    file3.seek(0)
                    json.dump(data3, file3)  
                    
                    
            import json
            rt ={
                    self._waypoint_count:rect,
                    
                }



            with open(r'C:\Users\Rakesh\Downloads\rectpositions.json','r+') as file6:
            
                    data6 = json.load(file6)
                    data6.update(rt)
                    file6.seek(0)
                    json.dump(data6, file6)   
                    
                    
            import json
            sd1 ={
                    self._waypoint_count:Scanning_direction_Waypoint_Distance,
                    
                }



            with open(r'C:\Users\Rakesh\Downloads\scanningwaypointdistance.json','r+') as file11:
            
                    data11 = json.load(file11)
                    data11.update(sd1)
                    file11.seek(0)
                    json.dump(data11, file11)                   
        

            # print("simulated_personposition",person)
            print("previous_blob_personposition",previous_blob_personposition)
            print("coverage",coverage)
            print("new_leader_index",new_leader_index)
            print("blobposition",blobposition)
            print("integralposition_centre",integralposition_centre)
            print("integral_startposition",integral_startposition)
            print("currentblob_personposition",blob_personposition)
            print("dronemeanpos",dronemeanpos)
            print("Scanning_direction",Scanning_direction)
            #print("personorientation",personorientation)
            # print("timeforcurrentiteration",person_time)
            print("Distancetravelledbyperson",person_distance)
            print("c3",Scanning_direction_Waypoint_Distance)
            
            
            motion = []
            motion.append('Waypoint ' + str(self._waypoint_count) + ' \n')
            motion.append('rect ' + str(rect) + ' \n')
            motion.append('previous_blob_personposition ' + str(previous_blob_personposition) + ' \n')
            motion.append('coverage ' + str(coverage) + ' \n')
            motion.append('new_leader_index ' + str(new_leader_index) + ' \n')
            motion.append('blobposition ' + str(blobposition) + ' \n')
            motion.append('integralposition_centre ' + str(integralposition_centre) + ' \n')
            motion.append('integral_startposition ' + str(integral_startposition) + ' \n')
            motion.append('currentblob_personposition ' + str(blob_personposition) + ' \n')
            motion.append('dronemeanpos ' + str(dronemeanpos) + ' \n')
            motion.append('Scanning_direction ' + str(Scanning_direction) + ' \n')
            #motion.append('personorientation ' + str(personorientation) + ' \n')
            # motion.append('timeforcurrentiteration ' + str(person_time) + ' \n')
            motion.append('Distancetravelledbyperson ' + str(person_distance) + ' \n')
            motion.append('Scanning_direction_Waypoint_Distance_c3 ' + str(Scanning_direction_Waypoint_Distance) + ' \n')
            self.set_scanning_direction(Scanning_direction)
            
            self._blob_person_position = blob_personposition

        with open(os.path.join(Live_Debug_Path,'motion.txt'), 'a') as f:
            f.writelines('\n'.join(motion))
            

        drones_next_east_loc = new_drone_rel_pos[0,:] + self._center_east
        drones_next_north_loc = new_drone_rel_pos[1,:] + self._center_north

        drones_next_lat,drones_next_lon = utm.to_latlon(drones_next_east_loc,drones_next_north_loc, self._zone_number, self._zone_letter)
        info.append('drones next lat ' + str(drones_next_lat) + ' drones next lon ' + str(drones_next_lon)+ ' \n')
        drones_next_loc = zip(drones_next_lat, drones_next_lon)
        #print(list(drones_next_loc))
        self._current_gps_waypoints = drones_next_loc
        #datasets_folder_live_debug= os.path.join( r"E:\Droneswarms_Feb_clientserverdevelopment\live_debug" )
        with open(os.path.join(Live_Debug_Path,'WayPoint_Info.txt'), 'a') as f:
            f.writelines('\n'.join(info))
        return drones_next_loc, self._previous_drone_pos, integral_image,leader_contour_area, leader_index, prev_contour_area,camera_id,leader_blob_image, leader_rx_image,gen_integral_blob_img,rect,personorientation,self._blob_person_position

    def euclidean_distance_sqr(self,point1, point2):
        """ euclidean_distance_sqr([1,2],[2,4])
        5
        """
        return (point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2


    def column_based_sort(self,array, column=0):
        """
        column_based_sort([(5, 1), (4, 2), (3, 0)], 1)
        [(3, 0), (5, 1), (4, 2)]
        """
        return sorted(array, key=lambda x: x[column])


    def dis_between_closest_pair(self,points, points_counts, min_dis=float("inf")):
        """
        brute force approach to find distance between closest pair points

        Parameters :
        points, points_count, min_dis (list(tuple(int, int)), int, int)

        Returns :
        min_dis (float):  distance between closest pair of points

        dis_between_closest_pair([[1,2],[2,4],[5,7],[8,9],[11,0]],5)
        5

        """

        for i in range(points_counts - 1):
            for j in range(i + 1, points_counts):
                current_dis = self.euclidean_distance_sqr(points[i], points[j])
                if current_dis < min_dis:
                    min_dis = current_dis
        return min_dis


    def dis_between_closest_in_strip(self,points, points_counts, min_dis=float("inf")):
        """
        closest pair of points in strip

        Parameters :
        points, points_count, min_dis (list(tuple(int, int)), int, int)

        Returns :
        min_dis (float):  distance btw closest pair of points in the strip (< min_dis)

        dis_between_closest_in_strip([[1,2],[2,4],[5,7],[8,9],[11,0]],5)
        85
        """

        for i in range(min(6, points_counts - 1), points_counts):
            for j in range(max(0, i - 6), i):
                current_dis = self.euclidean_distance_sqr(points[i], points[j])
                if current_dis < min_dis:
                    min_dis = current_dis
        return min_dis



    def closest_pair_of_points_sqr(self,points_sorted_on_x, points_sorted_on_y, points_counts):
        """divide and conquer approach

        Parameters :
        points, points_count (list(tuple(int, int)), int)

        Returns :
        (float):  distance btw closest pair of points

        closest_pair_of_points_sqr([(1, 2), (3, 4)], [(5, 6), (7, 8)], 2)
        8
        """

        # base case
        if points_counts <= 3:
            return self.dis_between_closest_pair(points_sorted_on_x, points_counts)

        # recursion
        mid = points_counts // 2
        closest_in_left = self.closest_pair_of_points_sqr(
            points_sorted_on_x, points_sorted_on_y[:mid], mid
        )
        closest_in_right = self.closest_pair_of_points_sqr(
            points_sorted_on_y, points_sorted_on_y[mid:], points_counts - mid
        )
        closest_pair_dis = min(closest_in_left, closest_in_right)

        """
        cross_strip contains the points, whose Xcoords are at a
        distance(< closest_pair_dis) from mid's Xcoord
        """

        cross_strip = []
        for point in points_sorted_on_x:
            if abs(point[0] - points_sorted_on_x[mid][0]) < closest_pair_dis:
                cross_strip.append(point)

        closest_in_strip = self.dis_between_closest_in_strip(
            cross_strip, len(cross_strip), closest_pair_dis
        )
        return min(closest_pair_dis, closest_in_strip)


    def closest_pair_of_points(self,points, points_counts):
        """
        closest_pair_of_points([(2, 3), (12, 30)], len([(2, 3), (12, 30)]))
        28.792360097775937
        """
        points_sorted_on_x = self.column_based_sort(points, column=0)
        points_sorted_on_y = self.column_based_sort(points, column=1)
        return (
            self.closest_pair_of_points_sqr(
                points_sorted_on_x, points_sorted_on_y, points_counts
            )
        ) ** 0.5



    def perpendicular( self, a ) :
        b = np.empty_like(a)
        b[0] = -a[1]
        b[1] = a[0]
        return b

    def normalize(self,a):
        a = np.array(a)
        return a/np.linalg.norm(a)

        
    def dis_between_farthest_pair(self,points, points_counts, min_dis=float(0)):
        """
        brute force approach to find distance between closest pair points

        Parameters :
        points, points_count, min_dis (list(tuple(int, int)), int, int)

        Returns :
        min_dis (float):  distance between closest pair of points

        dis_between_closest_pair([[1,2],[2,4],[5,7],[8,9],[11,0]],5)
        5

        """

        for i in range(points_counts - 1):
            for j in range(i + 1, points_counts):
                current_dis = self.euclidean_distance_sqr(points[i], points[j])
                if current_dis > min_dis:
                    min_dis = current_dis
        return min_dis

    def dis_between_farthest_in_strip(self,points, points_counts, min_dis=float(0)):
        """
        closest pair of points in strip

        Parameters :
        points, points_count, min_dis (list(tuple(int, int)), int, int)

        Returns :
        min_dis (float):  distance btw closest pair of points in the strip (< min_dis)

        dis_between_closest_in_strip([[1,2],[2,4],[5,7],[8,9],[11,0]],5)
        85
        """

        for i in range(min(6, points_counts - 1), points_counts):
            for j in range(max(0, i - 6), i):
                current_dis = self.euclidean_distance_sqr(points[i], points[j])
                if current_dis > min_dis:
                    min_dis = current_dis
        return min_dis

    def farthest_pair_of_points_sqr(self,points_sorted_on_x, points_sorted_on_y, points_counts):
        """divide and conquer approach

        Parameters :
        points, points_count (list(tuple(int, int)), int)

        Returns :
        (float):  distance btw closest pair of points

        closest_pair_of_points_sqr([(1, 2), (3, 4)], [(5, 6), (7, 8)], 2)
        8
        """

        # base case
        if points_counts <= 3:
            return self.dis_between_farthest_pair(points_sorted_on_x, points_counts)

        # recursion
        mid = points_counts // 2
        closest_in_left = self.closest_pair_of_points_sqr(
            points_sorted_on_x, points_sorted_on_y[:mid], mid
        )
        closest_in_right = self.closest_pair_of_points_sqr(
            points_sorted_on_y, points_sorted_on_y[mid:], points_counts - mid
        )
        closest_pair_dis = max(closest_in_left, closest_in_right)

        """
        cross_strip contains the points, whose Xcoords are at a
        distance(< closest_pair_dis) from mid's Xcoord
        """

        cross_strip = []
        for point in points_sorted_on_x:
            if abs(point[0] - points_sorted_on_x[mid][0]) > closest_pair_dis:
                cross_strip.append(point)

        closest_in_strip = self.dis_between_farthest_in_strip(
            cross_strip, len(cross_strip), closest_pair_dis
        )
        return max(closest_pair_dis, closest_in_strip)


    def farthest_pair_of_points(self,points, points_counts):
        """
        closest_pair_of_points([(2, 3), (12, 30)], len([(2, 3), (12, 30)]))
        28.792360097775937
        """
        points_sorted_on_x = self.column_based_sort(points, column=0)
        points_sorted_on_y = self.column_based_sort(points, column=1)
        return (
            self.farthest_pair_of_points_sqr(
                points_sorted_on_x, points_sorted_on_y, points_counts
            )
        ) ** 0.5

    def normalize(self,v):
        norm = np.linalg.norm(v)
        if norm == 0: 
            return v
        return v / norm

    def normalize_vectors(self,v):
        #print("normalizing vectors",v.shape[1])
        for i in range(v.shape[1]):
            temp = [v[0,i],v[1,i]]
            norm = np.linalg.norm(temp)
            if norm > 0.0: 
                v[0,i] = v[0,i]/norm
                v[1,i] = v[1,i]/norm
        return v

    def dist(self,p1, p2):
        return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)


    # def rutherford_scattering(self,Y,N,f):     # new_drone_rel_pos, self._no_of_drones, self._minimum_drone_distance
    #     ndim = 2
    #     masses = np.ones(N)                    
    #     charges = np.ones((N))*f
    #     loc_arr = np.transpose(Y) 
    #     speed_arr = np.zeros((N, ndim))

    #     # compute charge matrix, ie c1 * c2
    #     charge_matrix = -1 * np.outer(charges, charges)

    #     time = np.linspace(0, 0.5)
    #     dt = np.ediff1d(time).mean()

    #     for i, t in enumerate(time):
    #         # get (dx, dy) for every point
    #         delta = (loc_arr.T[..., np.newaxis] - loc_arr.T[:, np.newaxis]).T
    #         # calculate Euclidean distance
    #         distances = np.linalg.norm(delta, axis=-1)
    #         # and normalised unit vector
    #         unit_vector = (delta.T / distances).T
    #         unit_vector[np.isnan(unit_vector)] = 0 # replace NaN values with 0

    #         # calculate force
    #         force = charge_matrix / distances**2 # norm gives length of delta vector
    #         force[np.isinf(force)] = 0 # NaN forces are 0

    #         # calculate acceleration in all dimensions
    #         acc = (unit_vector.T * force / masses).T.sum(axis=1)
    #         # v = a * dt
    #         speed_arr += acc * dt

    #         # increment position, xyz = v * dt
    #         loc_arr += speed_arr * dt 

    #     return(np.transpose(loc_arr))
    
    
    def rutherford_scattering(self,Y,N,f):
        ndim = 2
    
        charges = np.ones((N))*f  #        charges = np.ones((N))*0.032*f

        loc_arr = np.transpose(Y) 
        speed_arr = np.zeros((N, ndim))

        # compute charge matrix, ie c1 * c2
        charge_matrix = -1 * np.outer(charges, charges)
        #print(charge_matrix)
        time = np.linspace(0, 0.5,500)
        dt = np.ediff1d(time).mean()
        masses = np.ones(N) *dt  #        masses = np.ones(N) *dt *dt


        for i, t in enumerate(time):
            # get (dx, dy) for every point
            delta = (loc_arr.T[..., np.newaxis] - loc_arr.T[:, np.newaxis]).T
            #print(delta)
            # calculate Euclidean distance
            distances = np.linalg.norm(delta, axis=-1)
            # and normalised unit vector
            unit_vector = (delta.T / distances).T
            unit_vector[np.isnan(unit_vector)] = 0 # replace NaN values with 0

            # calculate force
            force = charge_matrix / distances**2 # norm gives length of delta vector
            force[np.isinf(force)] = 0 # NaN forces are 0

            # calculate acceleration in all dimensions
            acc = (unit_vector.T * force / masses).T.sum(axis=1)
            # v = a * dt
            speed_arr = acc * dt

            # increment position, xyz = v * dt
            loc_arr += speed_arr * dt 

        return(np.transpose(loc_arr))

def eul2rotm(theta) :
    s_1 = math.sin(theta[0])
    c_1 = math.cos(theta[0]) 
    s_2 = math.sin(theta[1]) 
    c_2 = math.cos(theta[1]) 
    s_3 = math.sin(theta[2]) 
    c_3 = math.cos(theta[2])
    rotm = np.identity(3)
    rotm[0,0] =  c_1*c_2
    rotm[0,1] =  c_1*s_2*s_3 - s_1*c_3
    rotm[0,2] =  c_1*s_2*c_3 + s_1*s_3

    rotm[1,0] =  s_1*c_2
    rotm[1,1] =  s_1*s_2*s_3 + c_1*c_3
    rotm[1,2] =  s_1*s_2*c_3 - c_1*s_3

    rotm[2,0] = -s_2
    rotm[2,1] =  c_2*s_3
    rotm[2,2] =  c_2*c_3        

    return rotm


def imshow(image, *args, **kwargs):
     
    if len(image.shape) == 3: 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.imshow(image, *args, **kwargs)
    plt.axis('off')
    

def divide_by_alpha(img):
    a = img[:,:,3]
    aaa = np.stack((a,a,a),axis=-1)
    rgb = img[:,:,:3]/aaa 
    rgb[aaa==0] = np.nan
    return rgb

def divide_by_alpha1(rimg2):
    a = np.stack((rimg2[:,:,3],rimg2[:,:,3],rimg2[:,:,3]),axis=-1)
    return rimg2[:,:,:3]/a


def createviewmateuler(eulerang, camLocation):
    
    rotationmat = eul2rotm(eulerang)
    translVec =  np.reshape((-camLocation @ rotationmat),(3,1))
    conjoinedmat = (np.append(np.transpose(rotationmat), translVec, axis=1))
    return conjoinedmat


In [ ]:
#############################Definations defined above###############################################################
#############################Start the Chrome Browser###############################################################
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
options = Options()


options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable-automation"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
#############################Start the AOS Renderer###############################################################
w,h,fovDegrees = 512, 512, 50 # # resolution and field of view
render_fov = 50

if 'window' not in locals() or window == None:
                                    
    window = LFR.PyGlfwWindow( w, h, 'AOS' )  
     
aos = LFR.PyAOS(w,h,fovDegrees) 
set_folder = r'D:\Rendering\AOS-alpha_mask'
aos.loadDEM( os.path.join(set_folder,'zero_plane.obj'))


NumberofDrones = 10
fov = 50
rxthreshold = 0.9998
dem_height = 33
distance_btwn_drones = 4.2 #4.0 
Scanning_direction = 0
Scanning_direction_Waypoint_Distance = 2.0 #c3
emptyblobthreshold = 0.0
changing_to_linear_speed = 0.3 #c5
local_fac = 1.0   #c1
global_fac = 2.0   #c2  # global fac > local fac   # local fac == minimum distance
minimum_distance_btwn_drone = 4.2 #c4  4.0


person=[0,10]   #[8,21] (0,10), (-3,10)
personorientation = 100

import numpy as np

#prevwaypoints = np.array([[0,0,0,0,0,0,0,0,0,0],[-12,-12,-12,-12,-12,-12,-12,-12,-12,-12]], dtype = 'float')   # should be same as refloc


In [ ]:
#############################Get Drones Initial Positions###############################################################
test_drone_pso = drone_pso(aos,NumberofDrones,fov,rxthreshold,distance_btwn_drones,Scanning_direction,Scanning_direction_Waypoint_Distance,emptyblobthreshold,(48.335836, 14.326644),changing_to_linear_speed,local_fac,global_fac,minimum_distance_btwn_drone,False)
current_gps_locations,ref_loc = test_drone_pso.get_current_waypoints()

#for i in range (10):
#    ref_loc[1][i]= -20
    
# 48.335836, 14.326644
# added from n=10 sparse t = 175 last iteration = 63 (or 125 in data.json)

ref_loc =  np.array([[9.9717100350285, 7.509278207141478, 7.9509800794951735, 10.435511445470253, -0.7602028575178091, 13.721811741972076, 1.924510020547081, 4.666728601049116, 3.231715985565085, 0.936382834234614], [-8.656215061944506, -5.710321502200583, -0.16034449090046393, 3.5812591450458786, 0.30581173782537846, -5.231449516231961, -7.244081751496854, -2.52686032502034, 3.1534761204635524, -3.2818597748696416]], dtype = 'float')

###below is for old drones n=10 300trees/ha without altitude bias
#ref_loc =  np.array([[1.8115134503385846, 10.133639847964734, 12.629432704779868, 11.903718818156282, 6.7633418126185845, 4.691795821233404, 5.602373545591946, 8.155933054264636, 7.186430847096473, 0.3061385423385319], [-5.02637415149285, 5.097264629755322, 1.958122637084436, -3.986459164351714, 7.253806446287938, 1.848477920955482, -6.393277670197103, 1.9794189449672637, -2.746389521970734, 1.4854112721404695]], dtype = 'float')
#print(ref_loc)


prevwaypoints = ref_loc

#print(prevwaypoints)
drone_names = ['1','2','3','4','5','6','7','8','9','10']

#############################For First Step --- Determining Person Blob Size###############################################################
step_count = 0
#############################Create Link for Initial Positions###############################################################
altitude_list = [43,41,39,37,35,36,38,40,42,44] # [40,40,40,40,40,40,40,40,40,40] 
#### For 10 drones
link1="http://127.0.0.1:5504/?drone1x="+str(ref_loc[0][0])+"&drone1y="+str(ref_loc[1][0])+"&drone2x="+str(ref_loc[0][1])+"&drone2y="+str(ref_loc[1][1])+"&drone3x="+str(ref_loc[0][2])+"&drone3y="+str(ref_loc[1][2])+"&drone4x="+str(ref_loc[0][3])+"&drone4y="+str(ref_loc[1][3])+"&drone5x="+str(ref_loc[0][4])+"&drone5y="+str(ref_loc[1][4])+"&drone6x="+str(ref_loc[0][5])+"&drone6y="+str(ref_loc[1][5])+"&drone7x="+str(ref_loc[0][6])+"&drone7y="+str(ref_loc[1][6])+"&drone8x="+str(ref_loc[0][7])+"&drone8y="+str(ref_loc[1][7])+"&drone9x="+str(ref_loc[0][8])+"&drone9y="+str(ref_loc[1][8])+"&drone10x="+str(ref_loc[0][9])+"&drone10y="+str(ref_loc[1][9])+"&personx="+str(person[0])+"&persony="+str(person[1])+"&personorient="+str(personorientation)+"&filename="+('Waypoint'+str(step_count))+"&xlist="+(treex)+"&ylist="+(treey)+"&prevdrone1x="+str(prevwaypoints[0][0])+"&prevdrone1y="+str(prevwaypoints[1][0])+"&prevdrone2x="+str(prevwaypoints[0][1])+"&prevdrone2y="+str(prevwaypoints[1][1])+"&prevdrone3x="+str(prevwaypoints[0][2])+"&prevdrone3y="+str(prevwaypoints[1][2])+"&prevdrone4x="+str(prevwaypoints[0][3])+"&prevdrone4y="+str(prevwaypoints[1][3])+"&prevdrone5x="+str(prevwaypoints[0][4])+"&prevdrone5y="+str(prevwaypoints[1][4])+"&prevdrone6x="+str(prevwaypoints[0][5])+"&prevdrone6y="+str(prevwaypoints[1][5])+"&prevdrone7x="+str(prevwaypoints[0][6])+"&prevdrone7y="+str(prevwaypoints[1][6])+"&prevdrone8x="+str(prevwaypoints[0][7])+"&prevdrone8y="+str(prevwaypoints[1][7])+"&prevdrone9x="+str(prevwaypoints[0][8])+"&prevdrone9y="+str(prevwaypoints[1][8])+"&prevdrone10x="+str(prevwaypoints[0][9])+"&prevdrone10y="+str(prevwaypoints[1][9])+"&altd1="+str(altitude_list[0])+"&altd2="+str(altitude_list[1])+"&altd3="+str(altitude_list[2])+"&altd4="+str(altitude_list[3])+"&altd5="+str(altitude_list[4])+"&altd6="+str(altitude_list[5])+"&altd7="+str(altitude_list[6])+"&altd8="+str(altitude_list[7])+"&altd9="+str(altitude_list[8])+"&altd10="+str(altitude_list[9])

print(link1)
Saved_Img_Location = "Waypoint"+str(step_count) + '.'+'zip'
print(Saved_Img_Location)
print(os.path.splitext(Saved_Img_Location)[0])
Current_Waypoint_Images_Loc = os.path.join(Download_Location,'images', os.path.splitext(Saved_Img_Location)[0])
os.mkdir(Current_Waypoint_Images_Loc)
Current_Path = os.path.join(Download_Location,Saved_Img_Location)
print(Current_Path)
#############################Create Poses for Initial Positions###############################################################
#############################Create Poses for Initial Positions###############################################################
site_poses = []
for i in range(NumberofDrones):
    EastCentered = (ref_loc[0][i] - 0.0) #Get MeanEast and Set MeanEast
    NorthCentered = (0.0 - ref_loc[1][i]) #Get MeanNorth and Set MeanNorth
    Alt = dem_height
    M = createviewmateuler(np.array([0.0, 0.0, 0.0]),np.array( [ref_loc[0][i], ref_loc[1][i], 35 - altitude_list[i]] ))
    ViewMatrix = np.vstack((M, np.array([0.0,0.0,0.0,1.0],dtype=np.float32)))
    camerapose = np.asarray(ViewMatrix.transpose(),dtype=np.float32)
    site_poses.append(camerapose)  
#############################Send the Link to Simulator to generate the images###############################################################
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
options = Options()


options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable-automation"])
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()
driver.get(link1)

time.sleep(10)
while not os.path.exists(Current_Path):
    time.sleep(1)

if os.path.isfile(Current_Path):
    print("read file")
    with ZipFile(Current_Path,'r') as zipObj:
        zipObj.extractall(path=Current_Path.split(".")[0])
else:
    print('error')#
Current_Path = os.path.join(Download_Location,os.path.splitext(Saved_Img_Location)[0])
#############################Read the generated images###############################################################
#imagelist = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(top_path, '*.png')))]
single_images = []
for i in range(NumberofDrones):
    image_name = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(Current_Path, drone_names[i], 'camera', '*.png')))]
    g_1 = cv2.imread(os.path.join(Current_Path, drone_names[i], 'camera',image_name[0]))
    g_2 = cv2.imread(os.path.join(Current_Path, drone_names[i], 'camera',image_name[1]),cv2.IMREAD_GRAYSCALE)
    result = np.dstack((g_1, g_2))
    print('result: ', result.shape)
    #os.remove(os.path.join(Current_Path, drone_names[i], 'camera',image_name[0])) 
    #os.remove(os.path.join(Current_Path, drone_names[i], 'camera',image_name[1])) 
    cv2.imwrite(os.path.join(Current_Path, drone_names[i], 'camera','test'+str(i)+'.png'),result)
  
    
for i in range(NumberofDrones):
    image_name = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(Current_Path, drone_names[i], 'camera', '*.png')))]
    # print(Current_Path)
    # print(drone_names[i])
    print('camera',image_name[0]) 
    # print(Current_Waypoint_Images_Loc)
    # print(os.path.join(Current_Path, drone_names[i], 'camera',image_name[0]))
    shutil.copy2(os.path.join(Current_Path, drone_names[i], 'camera',image_name[2]), os.path.join(Current_Waypoint_Images_Loc,image_name[2]))
    img = cv2.imread(os.path.join(Current_Path, drone_names[i], 'camera',image_name[2]),cv2.IMREAD_UNCHANGED)
    print('Image Dimensions :', img.shape)
    single_images.append(img)
print(os.path.splitext(Saved_Img_Location)[0])
shutil.copy2(os.path.join(Current_Path, 'stage', 'cropped_image.png'), os.path.join(Stages_Path,os.path.splitext(Saved_Img_Location)[0]+'.png'))
print(prevwaypoints)
#############################For the First Step ---- Compute the Blob Threshold###############################################################

In [ ]:
#############################For the First Step ---- Compute the Blob Threshold###############################################################


fov = 50
rxthreshold = 0.9998
dem_height = 33
compasscorrection = 0
center_rgb_integral_image, leader_rx_image, Leader_index, leader_contour = test_drone_pso.determine_emptyblob_threshold(single_images,site_poses, fov = fov, rxthreshold =  rxthreshold, dem_height = dem_height, compasscorrection = compasscorrection , pos = None)
cv2.imwrite("rendered_image_setting_threshold.png",center_rgb_integral_image)
cv2.imwrite("leaderrx_image_setting_threshold.png",leader_rx_image)
print(leader_contour)
print(Leader_index)




#############################For the First Step ---- set the Blob Threshold###############################################################

In [ ]:
emptyblobthreshold = 220 #145  #170
test_drone_pso.set_emptyblob_threshold(emptyblobthreshold)

In [ ]:
person=[0,10]  #[15,21]
personorientation = 100
######################################Change Person Location After Initialization##########################################################
step_count = 1
print(ref_loc)
print(prevwaypoints)
altitude_list = [43,41,39,37,35,36,38,40,42,44] # [40,40,40,40,40,40,40,40,40,40] 
drawing_current_location = ref_loc
drawing_previous_location = prevwaypoints
print(ref_loc)
print(prevwaypoints)
c = []
for i in range (0, NumberofDrones):
    distance = math.sqrt( ((ref_loc[0][i]-prevwaypoints[0][i])**2)+((ref_loc[1][i]-prevwaypoints[1][i])**2) )
    c.append(distance)
    
print(c)
max_distance =  max(c)    
print(max(c))





import json
import os




d = {
    1:ref_loc.tolist(),
    2:prevwaypoints.tolist()
}



json.dump(d,open(r'C:\Users\Rakesh\Downloads\data.json',"w"))


jd = {
    1:person,
    
}



json.dump(jd,open(r'C:\Users\Rakesh\Downloads\personpositions.json',"w"))

# i = 3
# a = {
#     i:prevwaypoints.tolist(),
#     4:prevwaypoints.tolist()
# }


po = {
    1:personorientation,
    
}



json.dump(po,open(r'C:\Users\Rakesh\Downloads\personorientation.json',"w"))

rt2 = {
    
    
}



json.dump(rt2,open(r'C:\Users\Rakesh\Downloads\rectpositions.json',"w"))

import json
bd ={
        
        
    }



json.dump(bd,open(r'C:\Users\Rakesh\Downloads\blobpositions.json',"w"))


import json
sd ={
        
        
    }



json.dump(sd,open(r'C:\Users\Rakesh\Downloads\scanningwaypointdistance.json',"w"))

print(ref_loc)






print(ref_loc)

link1="http://127.0.0.1:5504/?drone1x="+str(ref_loc[0][0])+"&drone1y="+str(ref_loc[1][0])+"&drone2x="+str(ref_loc[0][1])+"&drone2y="+str(ref_loc[1][1])+"&drone3x="+str(ref_loc[0][2])+"&drone3y="+str(ref_loc[1][2])+"&drone4x="+str(ref_loc[0][3])+"&drone4y="+str(ref_loc[1][3])+"&drone5x="+str(ref_loc[0][4])+"&drone5y="+str(ref_loc[1][4])+"&drone6x="+str(ref_loc[0][5])+"&drone6y="+str(ref_loc[1][5])+"&drone7x="+str(ref_loc[0][6])+"&drone7y="+str(ref_loc[1][6])+"&drone8x="+str(ref_loc[0][7])+"&drone8y="+str(ref_loc[1][7])+"&drone9x="+str(ref_loc[0][8])+"&drone9y="+str(ref_loc[1][8])+"&drone10x="+str(ref_loc[0][9])+"&drone10y="+str(ref_loc[1][9])+"&personx="+str(person[0])+"&persony="+str(person[1])+"&personorient="+str(personorientation)+"&filename="+('Waypoint'+str(step_count))+"&xlist="+(treex)+"&ylist="+(treey)+"&prevdrone1x="+str(prevwaypoints[0][0])+"&prevdrone1y="+str(prevwaypoints[1][0])+"&prevdrone2x="+str(prevwaypoints[0][1])+"&prevdrone2y="+str(prevwaypoints[1][1])+"&prevdrone3x="+str(prevwaypoints[0][2])+"&prevdrone3y="+str(prevwaypoints[1][2])+"&prevdrone4x="+str(prevwaypoints[0][3])+"&prevdrone4y="+str(prevwaypoints[1][3])+"&prevdrone5x="+str(prevwaypoints[0][4])+"&prevdrone5y="+str(prevwaypoints[1][4])+"&prevdrone6x="+str(prevwaypoints[0][5])+"&prevdrone6y="+str(prevwaypoints[1][5])+"&prevdrone7x="+str(prevwaypoints[0][6])+"&prevdrone7y="+str(prevwaypoints[1][6])+"&prevdrone8x="+str(prevwaypoints[0][7])+"&prevdrone8y="+str(prevwaypoints[1][7])+"&prevdrone9x="+str(prevwaypoints[0][8])+"&prevdrone9y="+str(prevwaypoints[1][8])+"&prevdrone10x="+str(prevwaypoints[0][9])+"&prevdrone10y="+str(prevwaypoints[1][9])+"&altd1="+str(altitude_list[0])+"&altd2="+str(altitude_list[1])+"&altd3="+str(altitude_list[2])+"&altd4="+str(altitude_list[3])+"&altd5="+str(altitude_list[4])+"&altd6="+str(altitude_list[5])+"&altd7="+str(altitude_list[6])+"&altd8="+str(altitude_list[7])+"&altd9="+str(altitude_list[8])+"&altd10="+str(altitude_list[9])+"&leader="+str(Leader_index)
# Leader_index
print(link1)
print(ref_loc)
print(prevwaypoints)
Saved_Img_Location = "Waypoint"+ str(step_count) + '.'+'zip'
print(Saved_Img_Location)
print(os.path.splitext(Saved_Img_Location)[0])
Current_Waypoint_Images_Loc = os.path.join(Download_Location,'images', os.path.splitext(Saved_Img_Location)[0])
os.mkdir(Current_Waypoint_Images_Loc)
Current_Path = os.path.join(Download_Location,Saved_Img_Location)
print(Current_Path)
#############################Create Poses for Initial Positions###############################################################
#############################Create Poses for Initial Positions###############################################################
site_poses = []
for i in range(NumberofDrones):
    EastCentered = (ref_loc[0][i] - 0.0) #Get MeanEast and Set MeanEast
    NorthCentered = (0.0 - ref_loc[1][i]) #Get MeanNorth and Set MeanNorth
    Alt = dem_height
    M = createviewmateuler(np.array([0.0, 0.0, 0.0]),np.array( [ref_loc[0][i], ref_loc[1][i],  35 - altitude_list[i]] ))
    ViewMatrix = np.vstack((M, np.array([0.0,0.0,0.0,1.0],dtype=np.float32)))
    camerapose = np.asarray(ViewMatrix.transpose(),dtype=np.float32)
    site_poses.append(camerapose)
#############################Send the Link to Simulator to generate the images###############################################################
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(link1)
driver.maximize_window()
time.sleep(10)
while not os.path.exists(Current_Path):
    time.sleep(1)

if os.path.isfile(Current_Path):
    print("read file")
    with ZipFile(Current_Path,'r') as zipObj:
        zipObj.extractall(path=Current_Path.split(".")[0])
else:
    print('error')#
Current_Path = os.path.join(Download_Location,os.path.splitext(Saved_Img_Location)[0])
#############################Read the generated images###############################################################
#imagelist = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(top_path, '*.png')))]
single_images = []

# thermal_images_list = []
# for i in range(NumberofDrones):
#     image_name = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(Current_Path, drone_names[i], 'camera', '*.png')))]
#     g_2 = cv2.imread(os.path.join(Current_Path, drone_names[i], 'camera',image_name[1]))
#     thermal_images_list.append(g_2)

for i in range(NumberofDrones):
    image_name = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(Current_Path, drone_names[i], 'camera', '*.png')))]
    g_1 = cv2.imread(os.path.join(Current_Path, drone_names[i], 'camera',image_name[0]))
    g_2 = cv2.imread(os.path.join(Current_Path, drone_names[i], 'camera',image_name[1]),cv2.IMREAD_GRAYSCALE)
    result = np.dstack((g_1, g_2))
    print('result: ', result.shape)
    #os.remove(os.path.join(Current_Path, drone_names[i], 'camera',image_name[0])) 
    #os.remove(os.path.join(Current_Path, drone_names[i], 'camera',image_name[1])) 
    cv2.imwrite(os.path.join(Current_Path, drone_names[i], 'camera','test'+str(i)+'.png'),result)

for i in range(NumberofDrones):
    image_name = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(Current_Path, drone_names[i], 'camera', '*.png')))]
    # print(Current_Path)
    # print(drone_names[i])
    # print('camera',image_name[0]) 
    # print(Current_Waypoint_Images_Loc)
    # print(os.path.join(Current_Path, drone_names[i], 'camera',image_name[0]))
    shutil.copy2(os.path.join(Current_Path, drone_names[i], 'camera',image_name[2]), os.path.join(Current_Waypoint_Images_Loc,image_name[2]))
    
    shutil.copy2(os.path.join(Current_Path, drone_names[i], 'camera',image_name[1]), os.path.join(Current_Waypoint_Images_Loc,image_name[1]))
    img = cv2.imread(os.path.join(Current_Path, drone_names[i], 'camera',image_name[2]),cv2.IMREAD_UNCHANGED)
    print('Image Dimensions :', img.shape)
    single_images.append(img)
print(os.path.splitext(Saved_Img_Location)[0])
shutil.copy2(os.path.join(Current_Path, 'stage', 'cropped_image.png'), os.path.join(Stages_Path,os.path.splitext(Saved_Img_Location)[0]+'.png'))

#############################For the First Step ---- Compute the Blob Threshold###############################################################





In [ ]:
#imgs = test_drone_pso._metric_cal.project_images_to_all(aos,img_list = thermal_images_list,pose_list=site_poses,fov = fov,center_camera_index=5,project_images=False)


In [ ]:
#print(len(imgs))
#cv2.imwrite(os.path.join(r'C:\Users\Rakesh\Downloads\Final_result', ('stuff_' + str(0) + '.png')),imgs[0])

In [ ]:

aos.clearViews();
print(ref_loc)
print(prevwaypoints)
prevwaypoints = ref_loc
x_axis_mean = np.mean(np.asarray(ref_loc[0][:]))
y_axis_mean = np.mean(np.asarray(ref_loc[1][:]))
Alts = dem_height

#############################For the First Step ---- Get the Next Locations###############################################################            
center_EastCentered = (x_axis_mean - 0.0) #Get MeanEast and Set MeanEast
center_NorthCentered = (0.0 - y_axis_mean) #Get MeanNorth and Set MeanNorth
#center_M = createviewmateuler(np.array([(0.0), 0, 0]),np.array( [center_EastCentered, center_NorthCentered, -Alts] ))
center_M = createviewmateuler(np.array([0.0, 0.0, 0.0]),np.array( [x_axis_mean, y_axis_mean, 0.0] ))
center_ViewMatrix = np.vstack((center_M, np.array([0.0,0.0,0.0,1.0],dtype=np.float32)))
center_camerapose = np.asarray(center_ViewMatrix.transpose(),dtype=np.float32)



newgps, newloc, new_integral_image, new_leader_contour,new_leader_index,metric_area,cameraid,leader_blob_image, leader_rx_image,gen_integral_blob_img,rect,personorientation,blobposition = test_drone_pso.get_waypoints_with_pso_motion(single_images,site_poses,center_camerapose) ## Change Virtual Camera Pose to Center Position
#rx_image_normalize = cv2.normalize(leader_rx_image.astype('float'), None, 0.0, 1.0, norm_type=cv2.NORM_MINMAX) #normalize the mask to be between 1 and 0
#cv2.imwrite(os.path.join( leader_rx_image_folder,  str(2)+ 'leader_rx_image.png'), ((rx_image_normalize)*255).astype(np.uint8))

################################################################################motion################################################################3

# previous_blob_personposition = person 
# coverage = 2*altitude_list[new_leader_index]*np.tan(np.deg2rad(25))
# resolution = 512
# metretopixels = coverage/resolution
# blob_posx = rect[0] + rect[2]/2
# blob_posy = rect[1] + rect[3]/2
# blob_posxinpixels = blob_posx * metretopixels
# blob_posyinpixels = blob_posy * metretopixels
# blobposition = [blob_posxinpixels,blob_posyinpixels]
# integralposition_centre = [ref_loc[0][new_leader_index], ref_loc[1][new_leader_index]]
# integral_startposition = [integralposition_centre[0] - coverage/2, integralposition_centre[1] - coverage/2]
# blob_personposition = [blobposition[0] - abs(integral_startposition[0]), blobposition[1] - abs(integral_startposition[1])]

# dronemeanx=(np.mean(ref_loc[0]))
# dronemeany=(np.mean(ref_loc[1]))
# dronemeanpos = [dronemeanx,dronemeany]
# dy = (-blob_personposition[1]-(-dronemeany))
# dx = (blob_personposition[0]-dronemeanx)

# Scanning_direction = np.arctan2(dy,dx)
# Scanning_direction = ((np.degrees(Scanning_direction))%360)+90
# print(Scanning_direction)
# if Scanning_direction == 0 or Scanning_direction>=360 :
#     Scanning_direction = 0                             #need to switch sign of yaxis positive is down and negative is up
#     print(Scanning_direction)


# person = [0,10]
# dyp = (-blob_personposition[1]-(-previous_blob_personposition[1]))
# dxp = (blob_personposition[0]-previous_blob_personposition[0])
# personorientation = np.arctan2(dyp,dxp)
# personorientation = ((np.degrees(personorientation))%360)+90
# print(personorientation)
# if personorientation == 0 or personorientation>=360 :
#     personorientation = 0                             #need to switch sign of yaxis positive is down and negative is up
#     print(personorientation)
    
# delta = 0.2    
# c3 = 2.0

# Scanning_direction_Waypoint_Distance = c3 + delta #c3   # first waypoint so no change in c3

# print("simulated_personposition",person)
# print("previous_blob_personposition",previous_blob_personposition)
# print("coverage",coverage)
# print("new_leader_index",new_leader_index)
# print("blobposition",blobposition)
# print("integralposition_centre",integralposition_centre)
# print("integral_startposition",integral_startposition)
# print("currentblob_personposition",blob_personposition)
# print("dronemeanpos",dronemeanpos)
# print("Scanning_direction",Scanning_direction)
# print("personorientation",personorientation)
# print("c3",Scanning_direction_Waypoint_Distance)

# motion = []
# motion.append('Waypoint ' + str(1) + ' \n')
# motion.append('person ' + str(person) + ' \n')
# motion.append('previous_blob_personposition ' + str(previous_blob_personposition) + ' \n')
# motion.append('coverage ' + str(coverage) + ' \n')
# motion.append('new_leader_index ' + str(new_leader_index) + ' \n')
# motion.append('blobposition ' + str(blobposition) + ' \n')
# motion.append('integralposition_centre ' + str(integralposition_centre) + ' \n')
# motion.append('integral_startposition ' + str(integral_startposition) + ' \n')
# motion.append('currentblob_personposition ' + str(blob_personposition) + ' \n')
# motion.append('dronemeanpos ' + str(dronemeanpos) + ' \n')
# motion.append('Scanning_direction ' + str(Scanning_direction) + ' \n')
# motion.append('personorientation ' + str(personorientation) + ' \n')
# motion.append('Scanning_direction_Waypoint_Distance_c3 ' + str(Scanning_direction_Waypoint_Distance) + ' \n')
# with open(os.path.join(Live_Debug_Path,'motion.txt'), 'a') as f:
#     f.writelines('\n'.join(motion))






####################################################################motion##############################################################################



gen_integral_blob_img_bw = gen_integral_blob_img > 1
leader_blob_image_bw = leader_blob_image > 1 
print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",rect)
print("yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy",new_leader_index)
cv2.imwrite(os.path.join( leader_blob_image_folder,  str(1)+ 'leader_blob_image.png'), np.asarray(leader_blob_image_bw*255,dtype=np.uint8))
cv2.imwrite(os.path.join( leader_rx_image_folder,  str(1)+ 'leader_rx_image.png'), leader_rx_image)
cv2.imwrite(os.path.join( gen_integral_blob_img_folder,  str(1)+ 'gen_integral_blob_img.png'), np.asarray(gen_integral_blob_img_bw*255,dtype=np.uint8))




print(newloc)
print(prevwaypoints)
print(new_leader_contour)
print(new_leader_index, cameraid)
print(metric_area)
drawing_leader_index = cameraid



v1 = {
    0:cameraid
    
}



json.dump(v1,open(r'C:\Users\Rakesh\Downloads\cameraid.json',"w"))


v12 = {
    0:new_leader_index
    
}



json.dump(v12,open(r'C:\Users\Rakesh\Downloads\leader.json',"w"))


v13 = {
    0:metric_area
    
}



json.dump(v13,open(r'C:\Users\Rakesh\Downloads\metric_area.json',"w"))


v14 = {
    0:new_leader_contour
    
}



json.dump(v14,open(r'C:\Users\Rakesh\Downloads\new_leader_contour.json',"w"))




# altitude_list = [35,36,37,38,39,40,41,42,43,44]
# site_poses = []
# for i in range(NumberofDrones):
#     EastCentered = (ref_loc[0][i] - 0.0) #Get MeanEast and Set MeanEast
#     NorthCentered = (0.0 - ref_loc[1][i]) #Get MeanNorth and Set MeanNorth
#     Alt = dem_height
#     M = createviewmateuler(np.array([0.0, 0.0, 0.0]),np.array( [ref_loc[0][i], ref_loc[1][i],  35 - altitude_list[i]] ))
#     ViewMatrix = np.vstack((M, np.array([0.0,0.0,0.0,1.0],dtype=np.float32)))
#     camerapose = np.asarray(ViewMatrix.transpose(),dtype=np.float32)
#     site_poses.append(camerapose)
    

# single_images = []
# for i in range(NumberofDrones):
#     image_name = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(Current_Path, drone_names[i], 'camera', '*.png')))]
#     # print(Current_Path)
#     # print(drone_names[i])
#     # print('camera',image_name[0]) 
#     # print(Current_Waypoint_Images_Loc)
#     # print(os.path.join(Current_Path, drone_names[i], 'camera',image_name[0]))
#     shutil.copy2(os.path.join(Current_Path, drone_names[i], 'camera',image_name[1]), os.path.join(Current_Waypoint_Images_Loc,image_name[1]))
#     img = cv2.imread(os.path.join(Current_Path, drone_names[i], 'camera',image_name[1]))
#     print('Image Dimensions :', img.shape)
#     single_images.append(img)

# def divide_by_alpha(img):
#     a = img[:,:,3]
#     aaa = np.stack((a,a,a),axis=-1)
#     rgb = img[:,:,:3]/aaa 
#     rgb[aaa==0] = np.nan
#     return rgb


# for i in range(NumberofDrones):
#     aos.addView(single_images[i], site_poses[i], "01")
# inversecamerapose = glm.inverse(site_poses[2])
# #ivp = glm.inverse(glm.transpose(inversecamerapose))
# Posvec = glm.vec3(inversecamerapose[3])
# Upvec = glm.vec3(inversecamerapose[1])
# FrontVec = glm.vec3(inversecamerapose[2])
# cameraviewarr = np.array(glm.lookAt(Posvec, Posvec + FrontVec, Upvec))
    
    
# aos.setDEMTransform([0,0,dem_height])
# #aos.setPoseCorrectionall(self._aos.getViews(),[0,0,0], [0,0, self._compass_correction])
# tmp_tmp_integral_img = aos.render(cameraviewarr, 50)
# tmp_integral_img = divide_by_alpha(tmp_tmp_integral_img)
# cv2.imwrite(os.path.join( Integral_Path,  str(step_count-1)+ '_thermal.png'),  np.asarray(tmp_integral_img,dtype=np.uint8))
aos.clearViews();


#############################For the Second Step ---- Generate the link from the new locations############################################################### 

In [ ]:
import matplotlib.pyplot as plt

iter=1
iter_list = []
time_list = []
metric_list = []
k = 0
#metric_list.append(metric_area)
metric_list.append(748.5)
iter_list.append(iter)

print(metric_list)
drone_speed = 10
if max_distance == 0:
    time_taken = 0
else:    

    time_taken = (max_distance/drone_speed)


previous_time = time_taken
time_list.append(time_taken)

#plt.axis([-10, 200, -40, 1500])
plt.axis([-0.5, 120, -40, 1500])
print(time_list)
print(metric_list)

v15 = {
    0:time_taken
    
}



json.dump(v15,open(r'C:\Users\Rakesh\Downloads\time_taken.json',"w"))
#plt.xlabel("Iteration", fontsize=14)
plt.xlabel("Time (seconds)", fontsize=14)

plt.ylabel("Metric", fontsize=14)


plt.plot(time_list, metric_list,'xb-', markersize = 6)



#plt.show()
save_results_to = r'C:/Users/Rakesh/Downloads/plots/'

plt.savefig(save_results_to +  'image' + str(k) + '.png')
print(metric_list[0])


In [ ]:
import cv2
import numpy as np
import os
iter = 1


stageimg = cv2.imread( r'C:\Users\Rakesh\Downloads\stages\Waypoint' + str(iter) + '.png')

img_folder = os.path.join( r'C:\Users\Rakesh\Downloads\images\Waypoint' + str(iter))
print(img_folder)
import re

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

imagelist = []

import glob

for img in sorted(glob.glob(r'C:\Users\Rakesh\Downloads\images\Waypoint' + str(iter) + '/*.png'),key=numericalSort):
    n= cv2.imread(img)
    imagelist.append(n)
    

#print(imagelist[0])

integral = []
integral_img = os.path.join(r'C:\Users\Rakesh\Downloads\integrals')
for imges in sorted(glob.glob(r'C:\Users\Rakesh\Downloads\integrals' + '/*.png'),key=numericalSort):
    r= cv2.imread(imges)
    integral.append(r)
    print('Image Dimensions :', r.shape)
    
# cv2.imwrite(os.path.join( leader_blob_image_folder,  str(1)+ 'leader_blob_image.png'), leader_blob_image)
# cv2.imwrite(os.path.join( leader_rx_image_folder,  str(1)+ 'leader_rx_image.png'), leader_rx_image)
# cv2.imwrite(os.path.join( gen_integral_blob_img_folder,  str(1)+ 'gen_integral_blob_img.png'), gen_integral_blob_img)        
       
    
    
#print(integral)    
    
from cmath import tan
import numpy as np


thermal_dimension = round((2*(altitude_list[drawing_leader_index])*0.4663*9.33))     # Fov is 50 so theta by 2 is 25  

print(thermal_dimension)

mask = np.ones((thermal_dimension,thermal_dimension,3), np.uint8)   #(should be changed based on the leader)
h, w , k = mask.shape
print(drawing_current_location[0][drawing_leader_index])
xoff = round((467 + 33 + (9.33 * drawing_current_location[0][drawing_leader_index]) - (thermal_dimension /2)))  # leader x position instead of 18
yoff = round((467 + 31 + (9.33 * drawing_current_location[1][drawing_leader_index]) - (thermal_dimension /2)))   # leader y position instead of zero
result = stageimg.copy()
result[yoff:yoff+h, xoff:xoff+w] = mask    

final = result.copy()
dim = (thermal_dimension,thermal_dimension)  #(should be changed based on the leader)
thermalresizeimg = cv2.resize(integral[3], dim, interpolation = cv2.INTER_AREA)

h1, w1, k1 = thermalresizeimg.shape
print('resiyedimage',h1,w1,k1)
final[yoff:yoff+h1, xoff:xoff+w1] = thermalresizeimg
for i in range (NumberofDrones):
    x_currentposition = round(467 + 33 + ((275/(275-(altitude_list[i])))*9.33*drawing_current_location[0][i]))
    y_currentposition = round(467 + 31 + ((275/(275-(altitude_list[i])))*9.33*drawing_current_location[1][i]))
    x_previousposition = round(467 + 33 + ((275/(275-(altitude_list[i])))*9.33*drawing_previous_location[0][i]))
    y_previousposition = round(467 + 31 + ((275/(275-(altitude_list[i])))*9.33*drawing_previous_location[1][i]))
    print('x_currentposition',x_currentposition)
    print('y_currentposition',y_currentposition)
    print('x_previousposition',x_previousposition)
    print('y_previousposition',y_previousposition)
    


    cv2.circle(final, (x_currentposition, y_currentposition), 6, (255, 191, 0), -1)    # dimensions to be changed
    
     # dimensions to be changed

    leaderx_currentposition = round(467 + 33 + ((275/(275-(altitude_list[Leader_index])))*9.33*drawing_current_location[0][Leader_index]))
    leadery_currentposition = round(467 + 31 + ((275/(275-(altitude_list[Leader_index])))*9.33*drawing_current_location[1][Leader_index]))    
    cv2.circle(final, (leaderx_currentposition, leadery_currentposition), 6, (0, 255, 255), -1)   
    
    if (math.sqrt((x_previousposition-x_currentposition)**2 + (y_previousposition-y_currentposition)**2)) == 0 :
            
        division  = 0.1
    else:
        division  = (math.sqrt((x_previousposition-x_currentposition)**2 + (y_previousposition-y_currentposition)**2))
    cv2.arrowedLine(final, (x_previousposition, y_previousposition), (x_currentposition, y_currentposition), (255,255,255), 3, tipLength= 10/division)  
    
    
    
    
integral_metric = []
integral_metric_img = os.path.join(r'C:\Users\Rakesh\Downloads\plots')
for imges1 in sorted(glob.glob(r'C:\Users\Rakesh\Downloads\plots' + '/*.png'),key=numericalSort):
    r1= cv2.imread(imges1)
    integral_metric.append(r1)
    print('Image Dimensions :', r1.shape) 
    
    
    
leader_blob_image_list = []

for imges in sorted(glob.glob(r'C:\Users\Rakesh\Downloads\leader_blob_image' + '/*.png'),key=numericalSort):
    r2= cv2.imread(imges)
    leader_blob_image_list.append(r2)
    
leader_rx_image_list = []

for imges in sorted(glob.glob(r'C:\Users\Rakesh\Downloads\leader_rx_image' + '/*.png'),key=numericalSort):
    r3= cv2.imread(imges)
    leader_rx_image_list.append(r3)    
    
gen_integral_blob_img_list = []

for imges in sorted(glob.glob(r'C:\Users\Rakesh\Downloads\gen_integral_blob_img' + '/*.png'),key=numericalSort):
    r4= cv2.imread(imges)
    gen_integral_blob_img_list.append(r4)           


#print(integral[iter])
dim = (1536,1542)
dim1 =(256,256)
dim2 =(512,512)
white = [255,255,0]
rgbimg1= cv2.copyMakeBorder(imagelist[10],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
rgbimg6= cv2.copyMakeBorder(imagelist[15],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
stimg= cv2.copyMakeBorder(final,1,1,1,1,cv2.BORDER_CONSTANT,value=white)
therimg1= cv2.copyMakeBorder(imagelist[0],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
therimg2= cv2.copyMakeBorder(imagelist[1],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
therimg3= cv2.copyMakeBorder(imagelist[2],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
therimg4= cv2.copyMakeBorder(imagelist[3],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
therimg5= cv2.copyMakeBorder(imagelist[4],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
therimg6= cv2.copyMakeBorder(imagelist[5],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
therimg7= cv2.copyMakeBorder(imagelist[6],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
therimg8= cv2.copyMakeBorder(imagelist[7],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
therimg9= cv2.copyMakeBorder(imagelist[8],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
therimg10= cv2.copyMakeBorder(imagelist[9],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
rgbimg2= cv2.copyMakeBorder(imagelist[11],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
rgbimg3= cv2.copyMakeBorder(imagelist[12],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
rgbimg4= cv2.copyMakeBorder(imagelist[13],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
rgbimg5= cv2.copyMakeBorder(imagelist[14],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
rgbimg7= cv2.copyMakeBorder(imagelist[16],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
rgbimg8= cv2.copyMakeBorder(imagelist[17],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
rgbimg9= cv2.copyMakeBorder(imagelist[18],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
rgbimg10= cv2.copyMakeBorder(imagelist[19],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
irgb1= cv2.copyMakeBorder(integral[2],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
it1= cv2.copyMakeBorder(integral[3],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
met1= cv2.copyMakeBorder(integral_metric[0],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
a1 = cv2.copyMakeBorder(leader_blob_image_list[0],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
a2 = cv2.copyMakeBorder(leader_rx_image_list[0],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
a3 = cv2.copyMakeBorder(gen_integral_blob_img_list[0],1,1,1,1,cv2.BORDER_CONSTANT,value=white)

# img5= cv2.copyMakeBorder(imagelist[4],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
# img6= cv2.copyMakeBorder(imagelist[5],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
# img7= cv2.copyMakeBorder(imagelist[6],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
# img8= cv2.copyMakeBorder(imagelist[7],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
# img9= cv2.copyMakeBorder(imagelist[8],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
# img10= cv2.copyMakeBorder(imagelist[9],1,1,1,1,cv2.BORDER_CONSTANT,value=white)


# stage_img = cv2.resize(stageimg, dim, interpolation = cv2.INTER_AREA)
rgbimg1 = cv2.resize(rgbimg1, dim1, interpolation = cv2.INTER_AREA)
rgbimg6 = cv2.resize(rgbimg6, dim1, interpolation = cv2.INTER_AREA)
stimg = cv2.resize(stimg, dim, interpolation = cv2.INTER_AREA)
therimg1 = cv2.resize(therimg1, dim1, interpolation = cv2.INTER_AREA)
therimg2 = cv2.resize(therimg2, dim1, interpolation = cv2.INTER_AREA)
therimg3 = cv2.resize(therimg3, dim1, interpolation = cv2.INTER_AREA)
therimg4 = cv2.resize(therimg4, dim1, interpolation = cv2.INTER_AREA)
therimg5 = cv2.resize(therimg5, dim1, interpolation = cv2.INTER_AREA)
therimg6 = cv2.resize(therimg6, dim1, interpolation = cv2.INTER_AREA)
therimg7 = cv2.resize(therimg7, dim1, interpolation = cv2.INTER_AREA)
therimg8 = cv2.resize(therimg8, dim1, interpolation = cv2.INTER_AREA)
therimg9 = cv2.resize(therimg9, dim1, interpolation = cv2.INTER_AREA)
therimg10 = cv2.resize(therimg10, dim1, interpolation = cv2.INTER_AREA)

rgbimg2 = cv2.resize(rgbimg2, dim1, interpolation = cv2.INTER_AREA)
rgbimg3 = cv2.resize(rgbimg3, dim1, interpolation = cv2.INTER_AREA)
rgbimg4 = cv2.resize(rgbimg4, dim1, interpolation = cv2.INTER_AREA)
rgbimg5 = cv2.resize(rgbimg5, dim1, interpolation = cv2.INTER_AREA)
rgbimg7 = cv2.resize(rgbimg7, dim1, interpolation = cv2.INTER_AREA)
rgbimg8 = cv2.resize(rgbimg8, dim1, interpolation = cv2.INTER_AREA)
rgbimg9 = cv2.resize(rgbimg9, dim1, interpolation = cv2.INTER_AREA)
rgbimg10 = cv2.resize(rgbimg10, dim1, interpolation = cv2.INTER_AREA)
irgb1 = cv2.resize(irgb1, dim2, interpolation = cv2.INTER_AREA)
it1 = cv2.resize(it1, dim2, interpolation = cv2.INTER_AREA)
met1 = cv2.resize(met1, dim2, interpolation = cv2.INTER_AREA)
a1 = cv2.resize(a1, dim1, interpolation = cv2.INTER_AREA)
a2 = cv2.resize(a2, dim1, interpolation = cv2.INTER_AREA)
a3 = cv2.resize(a3, dim1, interpolation = cv2.INTER_AREA)
# img2 = cv2.resize(img2, dim1, interpolation = cv2.INTER_AREA)
# img3 = cv2.resize(img3, dim1, interpolation = cv2.INTER_AREA)
# img4 = cv2.resize(img4, dim1, interpolation = cv2.INTER_AREA)
# img5 = cv2.resize(img5, dim1, interpolation = cv2.INTER_AREA)
# img6 = cv2.resize(img6, dim1, interpolation = cv2.INTER_AREA)
# img7 = cv2.resize(img7, dim1, interpolation = cv2.INTER_AREA)
# img8 = cv2.resize(img8, dim1, interpolation = cv2.INTER_AREA)
# img9 = cv2.resize(img9, dim1, interpolation = cv2.INTER_AREA)
# img10 = cv2.resize(img10, dim1, interpolation = cv2.INTER_AREA)
# img11 = cv2.resize(integral_metric[k], dim1, interpolation = cv2.INTER_AREA)


h1, w1 = rgbimg1.shape[:2]
h6, w6 = rgbimg6.shape[:2]
sh1, sw1 = stimg.shape[:2]
h11, w11 = therimg1.shape[:2]
h12, w12 = therimg2.shape[:2]
h13, w13 = therimg3.shape[:2]
h14, w14 = therimg4.shape[:2]
h15, w15 = therimg5.shape[:2]
h16, w16 = therimg6.shape[:2]
h17, w17 = therimg7.shape[:2]
h18, w18 = therimg8.shape[:2]
h19, w19 = therimg9.shape[:2]
h20, w20 = therimg10.shape[:2]
h2, w2 = rgbimg2.shape[:2]
h3, w3 = rgbimg3.shape[:2]
h4, w4 = rgbimg4.shape[:2]
h5, w5 = rgbimg5.shape[:2]
h7, w7 = rgbimg7.shape[:2]
h8, w8 = rgbimg8.shape[:2]
h9, w9 = rgbimg9.shape[:2]
h10, w10 = rgbimg10.shape[:2]
ih1, iw1 = irgb1.shape[:2]
ith1, itw1 = it1.shape[:2]
meth1, metw1 = met1.shape[:2]
a1h, a1w = a1.shape[:2]
a2h, a2w = a2.shape[:2]
a3h, a3w = a3.shape[:2]

# h1, w1 = stage_img.shape[:2]
# h2, w2 = integral[iter].shape[:2]
# h3, w3 = img1.shape[:2]
# h4, w4 = img2.shape[:2]
# h5, w5 = img3.shape[:2]
# h6, w6 = img4.shape[:2]
# h7, w7 = img5.shape[:2]
# h8, w8 = img6.shape[:2]
# h9, w9 = img7.shape[:2]
# h10, w10 = img8.shape[:2]
# h11, w11 = img9.shape[:2]
# h12, w12 = img10.shape[:2]
# h13,w13 = img11.shape[:2]
# print(h12, w12)

img_3 = np.zeros((1542,3084,3), dtype=np.uint8)
# print(img_3[:h1, :w1,:3].shape)
# print(img_3[:h2, w1:w1+w2,:3].shape)
img_3[:h1, :w1,:3] = rgbimg1
img_3[:h1, w1:w1+w6,:3] = rgbimg6
img_3[:h6+h1+sh1, w1+w6:w1+w6+sw1 ,:3] = stimg
img_3[:h1, w1+w6+sw1:w1+w6+sw1+w11 ,:3] = therimg1
img_3[h1:h1+h2, w1+w6+sw1:w1+w6+sw1+w11 ,:3] = therimg2
img_3[h1+h2:h1+h2+h3, w1+w6+sw1:w1+w6+sw1+w11 ,:3] = therimg3
img_3[h1+h2+h3:h1+h2+h3+h4, w1+w6+sw1:w1+w6+sw1+w11 ,:3] = therimg4
img_3[h1+h2+h3+h4:h1+h2+h3+h4+h5, w1+w6+sw1:w1+w6+sw1+w11 ,:3] = therimg5
img_3[h1+h2+h3+h4+h5:h1+h2+h3+h4+h5+a1h, w1+w6+sw1:w1+w6+sw1+w11 ,:3] = a1

img_3[:h1, w1+w6+sw1+w11:w1+w6+sw1+w11+w16 ,:3] = therimg6
img_3[h1:h1+h2, w1+w6+sw1+w11:w1+w6+sw1+w11+w16 ,:3] = therimg7
img_3[h1+h2:h1+h2+h3, w1+w6+sw1+w11:w1+w6+sw1+w11+w16 ,:3] = therimg8
img_3[h1+h2+h3:h1+h2+h3+h4, w1+w6+sw1+w11:w1+w6+sw1+w11+w16 ,:3] = therimg9
img_3[h1+h2+h3+h4:h1+h2+h3+h4+h5, w1+w6+sw1+w11:w1+w6+sw1+w11+w16 ,:3] = therimg10
img_3[h1+h2+h3+h4+h5:h1+h2+h3+h4+h5+a3h, w1+w6+sw1+w11:w1+w6+sw1+w11+w16 ,:3] = a3
img_3[h1:h1+h2, :w2,:3] = rgbimg2
img_3[h1+h2:h1+h2+h3, :w3,:3] = rgbimg3
img_3[h1+h2+h3:h1+h2+h3+h4, :w2,:3] = rgbimg4
img_3[h1+h2+h3+h4:h1+h2+h3+h4+h5, :w2,:3] = rgbimg5
#img_3[h1+h2+h3+h4+h5:h1+h2+h3+h4+h5+a2h, :w2,:3] = a2
img_3[h1:h1+h2, w1:w1+w7,:3] = rgbimg7
img_3[h1+h2:h1+h2+h3, w1:w1+w8,:3] = rgbimg8
img_3[h1+h2+h3:h1+h2+h3+h4, w1:w1+w9,:3] = rgbimg9
img_3[h1+h2+h3+h4:h1+h2+h3+h4+h5, w1:w1+w10,:3] = rgbimg10
img_3[:h1+h6, w1+w6+sw1+w11+w16:w1+w6+sw1+w11+w16+iw1,:3] = irgb1
img_3[h1+h2:h1+h6+ith1, w1+w6+sw1+w11+w16:w1+w6+sw1+w11+w16+iw1,:3] = it1
img_3[h1+h2+ith1:h1+h6+ith1+meth1, w1+w6+sw1+w11+w16:w1+w6+sw1+w11+w16+iw1,:3] = met1
# img_3[:h2, w1:w1+w2,:3] = integral[iter]

# img_3[h1:h1+h3, :w3,:3] = img1
# img_3[h1:h1+h3, w3:w3+w4,:3] = img2
# img_3[h1:h1+h3, w2:w3+w4+w5,:3] = img3
# img_3[h1:h1+h3, w1:w3+w4+w5+w6,:3] = img4
# img_3[h1:h1+h3, w1+w3:w3+w4+w5+w6+w7,:3] = img5
# img_3[h1+h3:h1+h3+h4, :w8,:3] = img6
# img_3[h1+h3:h1+h3+h4, w8:w8+w9,:3] = img7
# img_3[h1+h3:h1+h3+h4, w8+w7:w8+w9+w10,:3] = img8
# img_3[h1+h3:h1+h3+h4, w8+w7+w9:w8+w9+w10+w11,:3] = img9
# img_3[h1+h3:h1+h3+h4, w8+w7+w9+w10:w8+w9+w10+w11+w12,:3] = img10
# img_3[h2:h2+h13, w1:w1+w13,:3] = img11


# #cv2.putText(img_3,"Metric:" + str(new_leader_contour), (830,615), cv2.FONT_HERSHEY_SIMPLEX, 1, 255)
# #cv2.putText(img_3,"Leader_index:" + str(new_leader_index), (820,660), cv2.FONT_HERSHEY_SIMPLEX, 1, 255)

# # print(img_3.shape)

# th = 0.1 # defines the value below which a pixel is considered "black"
# black_pixels = np.where(
#     (img_3[:, :, 0] < th) & 
#     (img_3[:, :, 1] < th) & 
#     (img_3[:, :, 2] < th)
# )
# img_3[black_pixels] = [255, 255, 255]

cv2.imwrite(os.path.join(r'C:\Users\Rakesh\Downloads\Final_result', ('stage_' + str(iter) + '.png')),img_3)

# iter = iter + 1

In [ ]:


ybefore = []
yafter = []
xbefore = []
xafter = []
ycamerabefore = []
ycameraafter = []
xcamerabefore = []
xcameraafter = []
cameralist = []
leaderlist = []
currentmeanx = []
currentmeany =[]
s = 1
o = 3
p = 4
start_time_person = 0
iter = 1
step_count = 1
old_theta = 270
for i in range(300):
    # if i==9 :
    #     emptyblobthreshold = 1000 #145  #170
    #     test_drone_pso.set_emptyblob_threshold(emptyblobthreshold)
    
    
    
        
    iter = iter + 1
    #personorientation = 0       
    
    aos.clearViews();
    #person=[0,10] 
   
    step_count = step_count + 1
    ref_loc = newloc
    #print(prevwaypoints)
    print(ref_loc)
    altitude_list = [43,41,39,37,35,36,38,40,42,44] #[40,40,40,40,40,40,40,40,40,40] 
    drawing_current_location = ref_loc
    drawing_previous_location = prevwaypoints
    c = []
    for i in range (0, NumberofDrones):
        distance = math.sqrt( ((ref_loc[0][i]-prevwaypoints[0][i])**2)+((ref_loc[1][i]-prevwaypoints[1][i])**2) )
        c.append(distance)
        
    print(c)
    new_max_distance =  max(c)    
    print(max(c))
    
    drone_totaltime = (new_max_distance/drone_speed)
    total_time = start_time_person + drone_totaltime
    
    
    
    # if iter <= 10:
    #     personorientation = 360
    #     px = total_time  * (math.sin(np.deg2rad(personorientation)))
    #     py = total_time  * (math.cos(np.deg2rad(personorientation))) #0.9848, 0.1736
   
    
    
    #  ####################################################################3
    
    #     person_speed = 4
    #     personinx = person_speed * px
    #     personiny = person_speed * py
    #     person = [((person[0]) + personinx), (person[1] + personiny)]
        
    # if iter>10 and iter<=50 :  
    #     person = person
    # if iter> 50:
    #     personorientation = 180
    #     px = total_time  * (math.sin(np.deg2rad(personorientation)))
    #     py = total_time  * (math.cos(np.deg2rad(personorientation))) #0.9848, 0.1736
   
    
    
    #  ####################################################################3
    
    #     person_speed = 4
    #     personinx = person_speed * px
    #     personiny = person_speed * py
    #     person = [((person[0]) + personinx), (person[1] + personiny)]
               
    
    
    
    
    
    
    
    
    #person = [0,10]



    
    
    
    
    
    
    ########################################################################3
   

    theta = 2*3.14*20
    theta = theta/360
    
    person_speed = 4
    personinx = person_speed * total_time
    personiny = person_speed * total_time
    
    new_theta = personinx / theta
    
    old_theta  = old_theta + new_theta
    
   
    
    x_dir = 20*np.cos(np.deg2rad(old_theta))
    y_dir = -10 + (-20*np.sin(np.deg2rad(old_theta)))
    
    new_person = [x_dir,y_dir]
    
    dx = new_person[0] - person[0]

    # Difference in y coordinates
    dy = -new_person[1] +person[1]

    # Angle between p1 and p2 in radians
    personorientation = math.atan2(dy, dx)
    personorientation = ((np.degrees(personorientation))%360)+90
    
    person = new_person
    
    
    
    ############################################################################
    
    
    
    a = {
        o:ref_loc.tolist(),
        p:prevwaypoints.tolist()
    }



    with open(r'C:\Users\Rakesh\Downloads\data.json' , 'r+') as file:
            data = json.load(file)
            data.update(a)
            file.seek(0)
            json.dump(data, file)   
            
            
    jd1 = {
        iter:person,
    
    }



    with open(r'C:\Users\Rakesh\Downloads\personpositions.json','r+') as file1:
            data1 = json.load(file1)
            data1.update(jd1)
            file1.seek(0)
            json.dump(data1, file1) 

    # i = 3
    # a = {
    #     i:prevwaypoints.tolist(),
    #     4:prevwaypoints.tolist()
    # }


    po1 = {
        iter:personorientation,
        
    }



    with open(r'C:\Users\Rakesh\Downloads\personorientation.json','r+') as file2: 
    
            data2 = json.load(file2)
            data2.update(po1)
            file2.seek(0)
            json.dump(data2, file2)      
        
    
    
    link1="http://127.0.0.1:5504/?drone1x="+str(ref_loc[0][0])+"&drone1y="+str(ref_loc[1][0])+"&drone2x="+str(ref_loc[0][1])+"&drone2y="+str(ref_loc[1][1])+"&drone3x="+str(ref_loc[0][2])+"&drone3y="+str(ref_loc[1][2])+"&drone4x="+str(ref_loc[0][3])+"&drone4y="+str(ref_loc[1][3])+"&drone5x="+str(ref_loc[0][4])+"&drone5y="+str(ref_loc[1][4])+"&drone6x="+str(ref_loc[0][5])+"&drone6y="+str(ref_loc[1][5])+"&drone7x="+str(ref_loc[0][6])+"&drone7y="+str(ref_loc[1][6])+"&drone8x="+str(ref_loc[0][7])+"&drone8y="+str(ref_loc[1][7])+"&drone9x="+str(ref_loc[0][8])+"&drone9y="+str(ref_loc[1][8])+"&drone10x="+str(ref_loc[0][9])+"&drone10y="+str(ref_loc[1][9])+"&personx="+str(person[0])+"&persony="+str(person[1])+"&personorient="+str(personorientation)+"&filename="+('Waypoint'+str(step_count))+"&xlist="+(treex)+"&ylist="+(treey)+"&prevdrone1x="+str(prevwaypoints[0][0])+"&prevdrone1y="+str(prevwaypoints[1][0])+"&prevdrone2x="+str(prevwaypoints[0][1])+"&prevdrone2y="+str(prevwaypoints[1][1])+"&prevdrone3x="+str(prevwaypoints[0][2])+"&prevdrone3y="+str(prevwaypoints[1][2])+"&prevdrone4x="+str(prevwaypoints[0][3])+"&prevdrone4y="+str(prevwaypoints[1][3])+"&prevdrone5x="+str(prevwaypoints[0][4])+"&prevdrone5y="+str(prevwaypoints[1][4])+"&prevdrone6x="+str(prevwaypoints[0][5])+"&prevdrone6y="+str(prevwaypoints[1][5])+"&prevdrone7x="+str(prevwaypoints[0][6])+"&prevdrone7y="+str(prevwaypoints[1][6])+"&prevdrone8x="+str(prevwaypoints[0][7])+"&prevdrone8y="+str(prevwaypoints[1][7])+"&prevdrone9x="+str(prevwaypoints[0][8])+"&prevdrone9y="+str(prevwaypoints[1][8])+"&prevdrone10x="+str(prevwaypoints[0][9])+"&prevdrone10y="+str(prevwaypoints[1][9])+"&altd1="+str(altitude_list[0])+"&altd2="+str(altitude_list[1])+"&altd3="+str(altitude_list[2])+"&altd4="+str(altitude_list[3])+"&altd5="+str(altitude_list[4])+"&altd6="+str(altitude_list[5])+"&altd7="+str(altitude_list[6])+"&altd8="+str(altitude_list[7])+"&altd9="+str(altitude_list[8])+"&altd10="+str(altitude_list[9])+"&leader="+str(new_leader_index)
    #print(link1)
    Saved_Img_Location = "Waypoint"+ str(step_count) + '.'+'zip'
   # print(Saved_Img_Location)
    #print(os.path.splitext(Saved_Img_Location)[0])
    Current_Waypoint_Images_Loc = os.path.join(Download_Location,'images', os.path.splitext(Saved_Img_Location)[0])
    os.mkdir(Current_Waypoint_Images_Loc)
    Current_Path = os.path.join(Download_Location,Saved_Img_Location)
    #print(Current_Path)
    #############################Create Poses for Initial Positions###############################################################
    #############################Create Poses for Initial Positions###############################################################
    site_poses = []
    for i in range(NumberofDrones):
        EastCentered = (ref_loc[0][i] - 0.0) #Get MeanEast and Set MeanEast
        NorthCentered = (0.0 - ref_loc[1][i]) #Get MeanNorth and Set MeanNorth
        Alt = dem_height
        M = createviewmateuler(np.array([0.0, 0.0, 0.0]),np.array( [ref_loc[0][i], ref_loc[1][i], 35 - altitude_list[i]] ))
        ViewMatrix = np.vstack((M, np.array([0.0,0.0,0.0,1.0],dtype=np.float32)))
        camerapose = np.asarray(ViewMatrix.transpose(),dtype=np.float32)
        site_poses.append(camerapose)
    #############################Send the Link to Simulator to generate the images###############################################################
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(link1)
    driver.maximize_window()
    time.sleep(10)
    while not os.path.exists(Current_Path):
        time.sleep(1)

    if os.path.isfile(Current_Path):
        #print("read file")
        with ZipFile(Current_Path,'r') as zipObj:
            zipObj.extractall(path=Current_Path.split(".")[0])
    else:
        print('error')
    Current_Path = os.path.join(Download_Location,os.path.splitext(Saved_Img_Location)[0])
    #############################Read the generated images###############################################################
    #imagelist = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(top_path, '*.png')))]
    single_images = []
    
    for i in range(NumberofDrones):
        image_name = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(Current_Path, drone_names[i], 'camera', '*.png')))]
        g_1 = cv2.imread(os.path.join(Current_Path, drone_names[i], 'camera',image_name[0]))
        g_2 = cv2.imread(os.path.join(Current_Path, drone_names[i], 'camera',image_name[1]),cv2.IMREAD_GRAYSCALE)
        result = np.dstack((g_1, g_2))
        #print('result: ', result.shape)
        #os.remove(os.path.join(Current_Path, drone_names[i], 'camera',image_name[0])) 
        #os.remove(os.path.join(Current_Path, drone_names[i], 'camera',image_name[1])) 
        cv2.imwrite(os.path.join(Current_Path, drone_names[i], 'camera','test'+str(i)+'.png'),result)
    
    for i in range(NumberofDrones):
        image_name = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(Current_Path, drone_names[i], 'camera', '*.png')))]
        # print(Current_Path)
        # print(drone_names[i])
        # print('camera',image_name[0]) 
        # print(Current_Waypoint_Images_Loc)
        # print(os.path.join(Current_Path, drone_names[i], 'camera',image_name[0]))
        shutil.copy2(os.path.join(Current_Path, drone_names[i], 'camera',image_name[2]), os.path.join(Current_Waypoint_Images_Loc,image_name[2]))
        shutil.copy2(os.path.join(Current_Path, drone_names[i], 'camera',image_name[1]), os.path.join(Current_Waypoint_Images_Loc,image_name[1]))
        img = cv2.imread(os.path.join(Current_Path, drone_names[i], 'camera',image_name[2]),cv2.IMREAD_UNCHANGED)
        single_images.append(img)
    #print(os.path.splitext(Saved_Img_Location)[0])
    shutil.copy2(os.path.join(Current_Path, 'stage', 'cropped_image.png'), os.path.join(Stages_Path,os.path.splitext(Saved_Img_Location)[0]+'.png'))
    #############################For the First Step ---- Compute the Blob Threshold###############################################################
    x_axis_mean = np.mean(np.asarray(ref_loc[0][:]))
    y_axis_mean = np.mean(np.asarray(ref_loc[1][:]))
    Alts = dem_height

    #############################For the First Step ---- Get the Next Locations###############################################################            
    center_EastCentered = (x_axis_mean - 0.0) #Get MeanEast and Set MeanEast
    center_NorthCentered = (0.0 - y_axis_mean) #Get MeanNorth and Set MeanNorth
    #center_M = createviewmateuler(np.array([(0.0), 0, 0]),np.array( [center_EastCentered, center_NorthCentered, -Alts] ))
    center_M = createviewmateuler(np.array([0.0, 0.0, 0.0]),np.array( [x_axis_mean, y_axis_mean, 0.0] ))
    center_ViewMatrix = np.vstack((center_M, np.array([0.0,0.0,0.0,1.0],dtype=np.float32)))
    center_camerapose = np.asarray(center_ViewMatrix.transpose(),dtype=np.float32)

    newgps, newloc, new_integral_image, new_leader_contour,new_leader_index,metric_area,cameraid,leader_blob_image, leader_rx_image,gen_integral_blob_img,rect,personorientation,blobposition = test_drone_pso.get_waypoints_with_pso_motion(single_images,site_poses,center_camerapose) ## Change Virtual Camera Pose to Center Position   #new_integral_img use this
    #print(newloc)
    
    ################################################################################motion################################################################3
    # if new_leader_contour > emptyblobthreshold:
    #     previous_blob_personposition = blob_personposition 
    #     coverage = 2*altitude_list[new_leader_index]*np.tan(np.deg2rad(25))
    #     resolution = 512
    #     metretopixels = coverage/resolution
    #     blob_posx = rect[0] + rect[2]/2
    #     blob_posy = rect[1] + rect[3]/2
    #     blob_posxinpixels = blob_posx * metretopixels
    #     blob_posyinpixels = blob_posy * metretopixels
    #     blobposition = [blob_posxinpixels,blob_posyinpixels]
    #     integralposition_centre = [ref_loc[0][new_leader_index], ref_loc[1][new_leader_index]]
    #     integral_startposition = [integralposition_centre[0] - coverage/2, integralposition_centre[1] - coverage/2]
    #     blob_personposition = [blobposition[0] - abs(integral_startposition[0]), blobposition[1] - abs(integral_startposition[1])]

    #     dronemeanx=(np.mean(ref_loc[0]))
    #     dronemeany=(np.mean(ref_loc[1]))
    #     dronemeanpos = [dronemeanx,dronemeany]
    #     dy = (-blob_personposition[1]-(-dronemeany))
    #     dx = (blob_personposition[0]-dronemeanx)

    #     Scanning_direction = np.arctan2(dy,dx)
    #     Scanning_direction = ((np.degrees(Scanning_direction))%360)+90
    #     print(Scanning_direction)
    #     if Scanning_direction == 0 or Scanning_direction>=360 :
    #         Scanning_direction = 0                             #need to switch sign of yaxis positive is down and negative is up
    #         print(Scanning_direction)


        
    #     dyp = (-blob_personposition[1]-(-previous_blob_personposition[1]))
    #     dxp = (blob_personposition[0]-previous_blob_personposition[0])
    #     personorientation = np.arctan2(dyp,dxp)
    #     personorientation = ((np.degrees(personorientation))%360)+90
    #     print(personorientation)
    #     if personorientation == 0 or personorientation>=360 :
    #         personorientation = 0                             #need to switch sign of yaxis positive is down and negative is up
    #         print(personorientation)
        
    #     delta = 0.2
    #     person_distance = math.sqrt( (blob_personposition[0] - previous_blob_personposition[0] )**2 + (blob_personposition[1] - previous_blob_personposition[1])**2 )
    #     person_time = drone_totaltime
    #     person_calculatedspeed = person_distance/person_time
    #     person_simspeed = 3    
    #     c3 = person_calculatedspeed *  person_time # to be updated
    #     Scanning_direction_Waypoint_Distance = c3 + delta #c3   

    #     print("simulated_personposition",person)
    #     print("previous_blob_personposition",previous_blob_personposition)
    #     print("coverage",coverage)
    #     print("new_leader_index",new_leader_index)
    #     print("blobposition",blobposition)
    #     print("integralposition_centre",integralposition_centre)
    #     print("integral_startposition",integral_startposition)
    #     print("currentblob_personposition",blob_personposition)
    #     print("dronemeanpos",dronemeanpos)
    #     print("Scanning_direction",Scanning_direction)
    #     print("personorientation",personorientation)
    #     print("timeforcurrentiteration",person_time)
    #     print("Distancetravelledbyperson",person_distance)
    #     print("c3",Scanning_direction_Waypoint_Distance)
        
        
    #     motion = []
    #     motion.append('Waypoint ' + str(iter) + ' \n')
    #     motion.append('person ' + str(person) + ' \n')
    #     motion.append('previous_blob_personposition ' + str(previous_blob_personposition) + ' \n')
    #     motion.append('coverage ' + str(coverage) + ' \n')
    #     motion.append('new_leader_index ' + str(new_leader_index) + ' \n')
    #     motion.append('blobposition ' + str(blobposition) + ' \n')
    #     motion.append('integralposition_centre ' + str(integralposition_centre) + ' \n')
    #     motion.append('integral_startposition ' + str(integral_startposition) + ' \n')
    #     motion.append('currentblob_personposition ' + str(blob_personposition) + ' \n')
    #     motion.append('dronemeanpos ' + str(dronemeanpos) + ' \n')
    #     motion.append('Scanning_direction ' + str(Scanning_direction) + ' \n')
    #     motion.append('personorientation ' + str(personorientation) + ' \n')
    #     motion.append('timeforcurrentiteration ' + str(person_time) + ' \n')
    #     motion.append('Distancetravelledbyperson ' + str(person_distance) + ' \n')
    #     motion.append('Scanning_direction_Waypoint_Distance_c3 ' + str(Scanning_direction_Waypoint_Distance) + ' \n')
    # else:
    #     motion.append('Waypoint ' + str(iter) + ' \n')
    #     motion.append('No convergence'  + ' \n') 
    #     motion.append('person ' + str(person) + ' \n')
    #     motion.append('No convergence'  + ' \n')  
    #     motion.append('Scanning_direction ' + str(Scanning_direction) + ' \n')
    #     motion.append('personorientation ' + str(personorientation) + ' \n')
        
    #     motion.append('Distancetravelledbyperson ' + str(person_distance) + ' \n')
    #     motion.append('Scanning_direction_Waypoint_Distance_c3 ' + str(Scanning_direction_Waypoint_Distance) + ' \n')
        
    # with open(os.path.join(Live_Debug_Path,'motion.txt'), 'a') as f:
    #     f.writelines('\n'.join(motion))


    ####################################################################motion##############################################################################
    
   
    
    gen_integral_blob_img_bw = gen_integral_blob_img > 1
    leader_blob_image_bw = leader_blob_image > 1 
    cv2.imwrite(os.path.join( leader_blob_image_folder,  str(s+1)+ 'leader_blob_image.png'), np.asarray(leader_blob_image_bw*255,dtype=np.uint8))
    cv2.imwrite(os.path.join( leader_rx_image_folder,  str(s+1)+ 'leader_rx_image.png'), leader_rx_image)
    cv2.imwrite(os.path.join( gen_integral_blob_img_folder,  str(s+1)+ 'gen_integral_blob_img.png'), np.asarray(gen_integral_blob_img_bw*255,dtype=np.uint8))
    xafter.append(ref_loc)
    yafter.append(ref_loc)
    xbefore.append(prevwaypoints)
    ybefore.append(prevwaypoints)
    cameralist.append(cameraid)
    #####################################3 following change to cameraid or new_leader_index 
    leaderlist.append(cameraid)
    
    
    xcameraafter.append(ref_loc[0][cameraid])
    ycameraafter.append(ref_loc[1][cameraid])
    xcamerabefore.append(prevwaypoints[0][cameraid])
    ycamerabefore.append(prevwaypoints[1][cameraid])
    ############################################ for means centre of the pack
    
    currentmeanx.append(np.mean(ref_loc[0]))
    currentmeany.append(np.mean(ref_loc[1]))
    
    
    
    
    #########################################
    
    prevwaypoints = ref_loc
    #print(prevwaypoints)
    print(new_leader_index, cameraid)
    print(new_leader_contour)
    print(metric_area)
    drawing_leader_index = cameraid
   
    v16 = {
         o-1:new_leader_index,
        
    }



    with open(r'C:\Users\Rakesh\Downloads\leader.json' , 'r+') as file:
            data1 = json.load(file)
            data1.update(v16)
            file.seek(0)
            json.dump(data1, file)      
            
    v17 = {
         o-1:cameraid,
        
    }



    with open(r'C:\Users\Rakesh\Downloads\cameraid.json' , 'r+') as file:
            data2 = json.load(file)
            data2.update(v17)
            file.seek(0)
            json.dump(data2, file)      
            
            
    v18 = {
         o-1:metric_area,
        
    }



    with open(r'C:\Users\Rakesh\Downloads\metric_area.json' , 'r+') as file:
            data3 = json.load(file)
            data3.update(v18)
            file.seek(0)
            json.dump(data3, file)              
                          
                          
    v19 = {
         o-1:new_leader_contour,
        
    }



    with open(r'C:\Users\Rakesh\Downloads\new_leader_contour.json' , 'r+') as file:
            data4 = json.load(file)
            data4.update(v19)
            file.seek(0)
            json.dump(data4, file)                
            
    
    
    import matplotlib.pyplot as plt
    
    
    
    metric_list.append(metric_area)
    iter_list.append(iter)
    print(metric_list)
    

    time_taken = (new_max_distance/drone_speed)
    previous_time = previous_time + time_taken
    time_list.append(previous_time)
    
    v20 = {
         o-1:previous_time,
        
    }



    with open(r'C:\Users\Rakesh\Downloads\time_taken.json' , 'r+') as file:
            data5 = json.load(file)
            data5.update(v20)
            file.seek(0)
            json.dump(data5, file)   
    
    #plt.axis([-10, 200, -40, 1500])
    plt.axis([-0.5, 120, -40, 1500])
    #plt.xlabel("Iteration", fontsize=14)
    plt.xlabel("Time (seconds)", fontsize=14)
    plt.ylabel("Metric", fontsize=14)
    plt.plot(time_list, metric_list,'xb-',markersize = 6)
    
    print(time_taken)
    print(time_list)
    print(metric_list)

    

    #plt.show()
    save_results_to = r'C:/Users/Rakesh/Downloads/plots/'

    plt.savefig(save_results_to +  'image' + str(iter-1) + '.png')
    print(metric_list)
    
    aos.clearViews();
    
    
    

    #############################For the Second Step ---- Generate the link from the new locations############################################################### 




    import cv2
    import numpy as np
    import os
    


    stageimg = cv2.imread( r'C:\Users\Rakesh\Downloads\stages\Waypoint' + str(iter) + '.png')

    img_folder = os.path.join( r'C:\Users\Rakesh\Downloads\images\Waypoint' + str(iter))
    #print(img_folder)
    import re

    numbers = re.compile(r'(\d+)')
    def numericalSort(value):
        parts = numbers.split(value)
        parts[1::2] = map(int, parts[1::2])
        return parts

    imagelist = []

    import glob

    for img in sorted(glob.glob(r'C:\Users\Rakesh\Downloads\images\Waypoint' + str(iter) + '/*.png'),key=numericalSort):
        n= cv2.imread(img)
        imagelist.append(n)
        

    #print(imagelist[0])

    integral = []
    integral_img = os.path.join(r'C:\Users\Rakesh\Downloads\integrals')
    for imges in sorted(glob.glob(r'C:\Users\Rakesh\Downloads\integrals' + '/*.png'),key=numericalSort):
        r= cv2.imread(imges)
        integral.append(r)
        #print('Image Dimensions :', r.shape)
        
        
    thermal_dimension = round((2*(altitude_list[drawing_leader_index])*0.4663*9.33))     # Fov is 50 so theta by 2 is 25  

    print(thermal_dimension)

    mask = np.ones((thermal_dimension,thermal_dimension,3), np.uint8)   #(should be changed based on the leader)
    h, w , k = mask.shape
    print(drawing_current_location[0][drawing_leader_index])
    xoff = round((467 + 33 + (9.33 * drawing_current_location[0][drawing_leader_index]) - (thermal_dimension /2)))  # leader x position instead of 18
    yoff = round((467 + 31 + (9.33 * drawing_current_location[1][drawing_leader_index]) - (thermal_dimension /2)))   # leader y position instead of zero
    result = stageimg.copy()
    result[yoff:yoff+h, xoff:xoff+w] = mask    

    final = result.copy()
    dim = (thermal_dimension,thermal_dimension)  #(should be changed based on the leader)
    thermalresizeimg = cv2.resize(integral[(iter*3)+(iter-1)], dim, interpolation = cv2.INTER_AREA)

    h1, w1, k1 = thermalresizeimg.shape
    print('resiyedimage',h1,w1,k1)
    final[yoff:yoff+h1, xoff:xoff+w1] = thermalresizeimg
    
    # if s >= 2:
    #     a = 0
        
    #     for m in range (len(xbefore) - 1):
    #         for k in range (10):      #remove this
                        
    #             x_1 = round(467 + 33 + ((275/(275-(altitude_list[k])))*9.33*xafter[a][0][k]))   # k replace with a number as index for a drone 
    #             y_1 = round(467 + 31 + ((275/(275-(altitude_list[k])))*9.33*yafter[a][1][k]))
    #             x_0 = round(467 + 33 + ((275/(275-(altitude_list[k])))*9.33*xbefore[a][0][k]))
    #             y_0 = round(467 + 31 + ((275/(275-(altitude_list[k])))*9.33*ybefore[a][1][k]))
                
                    
                

    
    #             cv2.line(final,(x_0,y_0),(x_1,y_1),(255,0,0),2)
    #         a = a+1  #keep this
    
    if s>=2:    
        
        g = 2   
        h = 1 
        for m in range (len(xbefore) - 1):
                
            #for k in range (10):      #remove this
                        
                # x_1 = round(467 + 33 + ((275/(275-(altitude_list[leaderlist[1]])))*9.33*xcameraafter[h]))   # k replace with a number as index for a drone 
                # y_1 = round(467 + 31 + ((275/(275-(altitude_list[leaderlist[1]])))*9.33*ycameraafter[h]))
                # x_0 = round(467 + 33 + ((275/(275-(altitude_list[leaderlist[0]])))*9.33*xcameraafter[h-1]))
                # y_0 = round(467 + 31 + ((275/(275-(altitude_list[leaderlist[0]])))*9.33*ycameraafter[h-1]))
                # cv2.line(final,(x_0,y_0),(x_1,y_1),(255,0,0),2)
                
                x_1 = round(467 + 33 + ((275/(275-(np.mean(altitude_list))))*9.33*currentmeanx[h]))   # k replace with a number as index for a drone 
                y_1 = round(467 + 31 + ((275/(275-(np.mean(altitude_list))))*9.33*currentmeany[h]))
                x_0 = round(467 + 33 + ((275/(275-(np.mean(altitude_list))))*9.33*currentmeanx[h-1]))
                y_0 = round(467 + 31 + ((275/(275-(np.mean(altitude_list))))*9.33*currentmeany[h-1]))
                cv2.line(final,(x_0,y_0),(x_1,y_1),(255,0,0),2)
                
                
        for m in range (len(xbefore)  - 1 ):        
            if ((len(xbefore)) >=3  and  g < (len(xbefore))):
                    
                    # x_1 = round(467 + 33 + ((275/(275-(altitude_list[leaderlist[g]])))*9.33*xcameraafter[g]))   # k replace with a number as index for a drone 
                    # y_1 = round(467 + 31 + ((275/(275-(altitude_list[leaderlist[g]])))*9.33*ycameraafter[g]))
                    # x_0 = round(467 + 33 + ((275/(275-(altitude_list[leaderlist[g-1]])))*9.33*xcameraafter[g-1]))
                    # y_0 = round(467 + 31 + ((275/(275-(altitude_list[leaderlist[g-1]])))*9.33*ycameraafter[g-1]))
                    # g = g + 1
                    
                    x_1 = round(467 + 33 + ((275/(275-(np.mean(altitude_list))))*9.33*currentmeanx[g]))   # k replace with a number as index for a drone 
                    y_1 = round(467 + 31 + ((275/(275-(np.mean(altitude_list))))*9.33*currentmeany[g]))
                    x_0 = round(467 + 33 + ((275/(275-(np.mean(altitude_list))))*9.33*currentmeanx[g-1]))
                    y_0 = round(467 + 31 + ((275/(275-(np.mean(altitude_list))))*9.33*currentmeany[g-1]))
                    g = g + 1

    
                    cv2.line(final,(x_0,y_0),(x_1,y_1),(255,0,0),2)
    
    
    
    
    for k in range (NumberofDrones):
        x_currentposition = round(467 + 33 + ((275/(275-(altitude_list[k])))*9.33*drawing_current_location[0][k]))
        y_currentposition = round(467 + 31 + ((275/(275-(altitude_list[k])))*9.33*drawing_current_location[1][k]))
        x_previousposition = round(467 + 33 + ((275/(275-(altitude_list[k])))*9.33*drawing_previous_location[0][k]))
        y_previousposition = round(467 + 31 + ((275/(275-(altitude_list[k])))*9.33*drawing_previous_location[1][k]))


        cv2.circle(final, (x_currentposition, y_currentposition), 6, (255, 191, 0), -1)    # dimensions to be changed
        
        # dimensions to be changed

        leaderx_currentposition = round(467 + 33 + ((275/(275-(altitude_list[new_leader_index])))*9.33*drawing_current_location[0][new_leader_index]))
        leadery_currentposition = round(467 + 31 + ((275/(275-(altitude_list[new_leader_index])))*9.33*drawing_current_location[1][new_leader_index]))    
        cv2.circle(final, (leaderx_currentposition, leadery_currentposition), 6, (0, 255, 255), -1)   
        
        if (math.sqrt((x_previousposition-x_currentposition)**2 + (y_previousposition-y_currentposition)**2)) == 0 :
            
            division  = 0.1
        else:
            division  = (math.sqrt((x_previousposition-x_currentposition)**2 + (y_previousposition-y_currentposition)**2))  
            
       
        cv2.arrowedLine(final, (x_previousposition, y_previousposition), (x_currentposition, y_currentposition), (255,255,255), 3, tipLength= 10/division)    
        
    # if s >= 2:
    #     a = 0
        
    #     for m in range (len(xbefore) - 1):
    #         for k in range (10):      #remove this
                        
    #             x_1 = round(467 + 33 + ((275/(275-(altitude_list[k])))*9.33*xafter[a][0][k]))   # k replace with a number as index for a drone 
    #             y_1 = round(467 + 31 + ((275/(275-(altitude_list[k])))*9.33*yafter[a][1][k]))
    #             x_0 = round(467 + 33 + ((275/(275-(altitude_list[k])))*9.33*xbefore[a][0][k]))
    #             y_0 = round(467 + 31 + ((275/(275-(altitude_list[k])))*9.33*ybefore[a][1][k]))
                
                    
                

    
    #             cv2.line(final,(x_0,y_0),(x_1,y_1),(255,0,0),2)
    #         a = a+1  #keep this
    # if s>=2:    
        
    #     g = 2   
    #     h = 1 
    #     for m in range (len(xbefore) - 1):
                
    #         #for k in range (10):      #remove this
                        
    #             x_1 = round(467 + 33 + ((275/(275-(altitude_list[leaderlist[1]])))*9.33*xcameraafter[h]))   # k replace with a number as index for a drone 
    #             y_1 = round(467 + 31 + ((275/(275-(altitude_list[leaderlist[1]])))*9.33*ycameraafter[h]))
    #             x_0 = round(467 + 33 + ((275/(275-(altitude_list[leaderlist[0]])))*9.33*xcameraafter[h-1]))
    #             y_0 = round(467 + 31 + ((275/(275-(altitude_list[leaderlist[0]])))*9.33*ycameraafter[h-1]))
    #             cv2.line(final,(x_0,y_0),(x_1,y_1),(255,0,0),2)
                
    #     for m in range (len(xbefore)  - 1 ):        
    #         if ((len(xbefore)) >=3  and  g < (len(xbefore))):
                    
    #                 x_1 = round(467 + 33 + ((275/(275-(altitude_list[leaderlist[g]])))*9.33*xcameraafter[g]))   # k replace with a number as index for a drone 
    #                 y_1 = round(467 + 31 + ((275/(275-(altitude_list[leaderlist[g]])))*9.33*ycameraafter[g]))
    #                 x_0 = round(467 + 33 + ((275/(275-(altitude_list[leaderlist[g-1]])))*9.33*xcameraafter[g-1]))
    #                 y_0 = round(467 + 31 + ((275/(275-(altitude_list[leaderlist[g-1]])))*9.33*ycameraafter[g-1]))
    #                 g = g + 1

    
    #                 cv2.line(final,(x_0,y_0),(x_1,y_1),(255,0,0),2)
               
            
            
    integral_metric = []
    integral_metric_img = os.path.join(r'C:\Users\Rakesh\Downloads\plots')
    for imges1 in sorted(glob.glob(r'C:\Users\Rakesh\Downloads\plots' + '/*.png'),key=numericalSort):
        r1= cv2.imread(imges1)
        integral_metric.append(r1)
        #print('Image Dimensions :', r1.shape)  
        
    leader_blob_image_list = []

    for imges in sorted(glob.glob(r'C:\Users\Rakesh\Downloads\leader_blob_image' + '/*.png'),key=numericalSort):
        r2= cv2.imread(imges)
        leader_blob_image_list.append(r2)
        
    leader_rx_image_list = []

    for imges in sorted(glob.glob(r'C:\Users\Rakesh\Downloads\leader_rx_image' + '/*.png'),key=numericalSort):
        r3= cv2.imread(imges)
        leader_rx_image_list.append(r3)    
        
    gen_integral_blob_img_list = []

    for imges in sorted(glob.glob(r'C:\Users\Rakesh\Downloads\gen_integral_blob_img' + '/*.png'),key=numericalSort):
        r4= cv2.imread(imges)
        gen_integral_blob_img_list.append(r4)       


    #print(integral[iter])
    dim = (1536,1542)
    dim1 =(256,256)
    dim2 =(512,512)
    white = [255,255,0]
    rgbimg1= cv2.copyMakeBorder(imagelist[10],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    rgbimg6= cv2.copyMakeBorder(imagelist[15],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    stimg= cv2.copyMakeBorder(final,1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    therimg1= cv2.copyMakeBorder(imagelist[0],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    therimg2= cv2.copyMakeBorder(imagelist[1],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    therimg3= cv2.copyMakeBorder(imagelist[2],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    therimg4= cv2.copyMakeBorder(imagelist[3],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    therimg5= cv2.copyMakeBorder(imagelist[4],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    therimg6= cv2.copyMakeBorder(imagelist[5],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    therimg7= cv2.copyMakeBorder(imagelist[6],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    therimg8= cv2.copyMakeBorder(imagelist[7],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    therimg9= cv2.copyMakeBorder(imagelist[8],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    therimg10= cv2.copyMakeBorder(imagelist[9],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    rgbimg2= cv2.copyMakeBorder(imagelist[11],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    rgbimg3= cv2.copyMakeBorder(imagelist[12],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    rgbimg4= cv2.copyMakeBorder(imagelist[13],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    rgbimg5= cv2.copyMakeBorder(imagelist[14],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    rgbimg7= cv2.copyMakeBorder(imagelist[16],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    rgbimg8= cv2.copyMakeBorder(imagelist[17],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    rgbimg9= cv2.copyMakeBorder(imagelist[18],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    rgbimg10= cv2.copyMakeBorder(imagelist[19],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    irgb1= cv2.copyMakeBorder(integral[(iter*3)+(iter-2)],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    it1= cv2.copyMakeBorder(integral[(iter*3)+(iter-1)],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    met1= cv2.copyMakeBorder(integral_metric[iter-1],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    a1 = cv2.copyMakeBorder(leader_blob_image_list[s],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    a2 = cv2.copyMakeBorder(leader_rx_image_list[s],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    a3 = cv2.copyMakeBorder(gen_integral_blob_img_list[s],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    # img5= cv2.copyMakeBorder(imagelist[4],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    # img6= cv2.copyMakeBorder(imagelist[5],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    # img7= cv2.copyMakeBorder(imagelist[6],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    # img8= cv2.copyMakeBorder(imagelist[7],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    # img9= cv2.copyMakeBorder(imagelist[8],1,1,1,1,cv2.BORDER_CONSTANT,value=white)
    # img10= cv2.copyMakeBorder(imagelist[9],1,1,1,1,cv2.BORDER_CONSTANT,value=white)


    # stage_img = cv2.resize(stageimg, dim, interpolation = cv2.INTER_AREA)
    rgbimg1 = cv2.resize(rgbimg1, dim1, interpolation = cv2.INTER_AREA)
    rgbimg6 = cv2.resize(rgbimg6, dim1, interpolation = cv2.INTER_AREA)
    stimg = cv2.resize(stimg, dim, interpolation = cv2.INTER_AREA)
    therimg1 = cv2.resize(therimg1, dim1, interpolation = cv2.INTER_AREA)
    therimg2 = cv2.resize(therimg2, dim1, interpolation = cv2.INTER_AREA)
    therimg3 = cv2.resize(therimg3, dim1, interpolation = cv2.INTER_AREA)
    therimg4 = cv2.resize(therimg4, dim1, interpolation = cv2.INTER_AREA)
    therimg5 = cv2.resize(therimg5, dim1, interpolation = cv2.INTER_AREA)
    therimg6 = cv2.resize(therimg6, dim1, interpolation = cv2.INTER_AREA)
    therimg7 = cv2.resize(therimg7, dim1, interpolation = cv2.INTER_AREA)
    therimg8 = cv2.resize(therimg8, dim1, interpolation = cv2.INTER_AREA)
    therimg9 = cv2.resize(therimg9, dim1, interpolation = cv2.INTER_AREA)
    therimg10 = cv2.resize(therimg10, dim1, interpolation = cv2.INTER_AREA)

    rgbimg2 = cv2.resize(rgbimg2, dim1, interpolation = cv2.INTER_AREA)
    rgbimg3 = cv2.resize(rgbimg3, dim1, interpolation = cv2.INTER_AREA)
    rgbimg4 = cv2.resize(rgbimg4, dim1, interpolation = cv2.INTER_AREA)
    rgbimg5 = cv2.resize(rgbimg5, dim1, interpolation = cv2.INTER_AREA)
    rgbimg7 = cv2.resize(rgbimg7, dim1, interpolation = cv2.INTER_AREA)
    rgbimg8 = cv2.resize(rgbimg8, dim1, interpolation = cv2.INTER_AREA)
    rgbimg9 = cv2.resize(rgbimg9, dim1, interpolation = cv2.INTER_AREA)
    rgbimg10 = cv2.resize(rgbimg10, dim1, interpolation = cv2.INTER_AREA)
    irgb1 = cv2.resize(irgb1, dim2, interpolation = cv2.INTER_AREA)
    it1 = cv2.resize(it1, dim2, interpolation = cv2.INTER_AREA)
    met1 = cv2.resize(met1, dim2, interpolation = cv2.INTER_AREA)
    a1 = cv2.resize(a1, dim1, interpolation = cv2.INTER_AREA)
    a2 = cv2.resize(a2, dim1, interpolation = cv2.INTER_AREA)
    a3 = cv2.resize(a3, dim1, interpolation = cv2.INTER_AREA)
    # img2 = cv2.resize(img2, dim1, interpolation = cv2.INTER_AREA)
    # img3 = cv2.resize(img3, dim1, interpolation = cv2.INTER_AREA)
    # img4 = cv2.resize(img4, dim1, interpolation = cv2.INTER_AREA)
    # img5 = cv2.resize(img5, dim1, interpolation = cv2.INTER_AREA)
    # img6 = cv2.resize(img6, dim1, interpolation = cv2.INTER_AREA)
    # img7 = cv2.resize(img7, dim1, interpolation = cv2.INTER_AREA)
    # img8 = cv2.resize(img8, dim1, interpolation = cv2.INTER_AREA)
    # img9 = cv2.resize(img9, dim1, interpolation = cv2.INTER_AREA)
    # img10 = cv2.resize(img10, dim1, interpolation = cv2.INTER_AREA)
    # img11 = cv2.resize(integral_metric[k], dim1, interpolation = cv2.INTER_AREA)


    h1, w1 = rgbimg1.shape[:2]
    h6, w6 = rgbimg6.shape[:2]
    sh1, sw1 = stimg.shape[:2]
    h11, w11 = therimg1.shape[:2]
    h12, w12 = therimg2.shape[:2]
    h13, w13 = therimg3.shape[:2]
    h14, w14 = therimg4.shape[:2]
    h15, w15 = therimg5.shape[:2]
    h16, w16 = therimg6.shape[:2]
    h17, w17 = therimg7.shape[:2]
    h18, w18 = therimg8.shape[:2]
    h19, w19 = therimg9.shape[:2]
    h20, w20 = therimg10.shape[:2]
    h2, w2 = rgbimg2.shape[:2]
    h3, w3 = rgbimg3.shape[:2]
    h4, w4 = rgbimg4.shape[:2]
    h5, w5 = rgbimg5.shape[:2]
    h7, w7 = rgbimg7.shape[:2]
    h8, w8 = rgbimg8.shape[:2]
    h9, w9 = rgbimg9.shape[:2]
    h10, w10 = rgbimg10.shape[:2]
    ih1, iw1 = irgb1.shape[:2]
    ith1, itw1 = it1.shape[:2]
    meth1, metw1 = met1.shape[:2]
    a1h, a1w = a1.shape[:2]
    a2h, a2w = a2.shape[:2]
    a3h, a3w = a3.shape[:2]

    # h1, w1 = stage_img.shape[:2]
    # h2, w2 = integral[iter].shape[:2]
    # h3, w3 = img1.shape[:2]
    # h4, w4 = img2.shape[:2]
    # h5, w5 = img3.shape[:2]
    # h6, w6 = img4.shape[:2]
    # h7, w7 = img5.shape[:2]
    # h8, w8 = img6.shape[:2]
    # h9, w9 = img7.shape[:2]
    # h10, w10 = img8.shape[:2]
    # h11, w11 = img9.shape[:2]
    # h12, w12 = img10.shape[:2]
    # h13,w13 = img11.shape[:2]
    # print(h12, w12)

    img_3 = np.zeros((1542,3084,3), dtype=np.uint8)
    # print(img_3[:h1, :w1,:3].shape)
    # print(img_3[:h2, w1:w1+w2,:3].shape)
    img_3[:h1, :w1,:3] = rgbimg1
    img_3[:h1, w1:w1+w6,:3] = rgbimg6
    img_3[:h6+h1+sh1, w1+w6:w1+w6+sw1 ,:3] = stimg
    img_3[:h1, w1+w6+sw1:w1+w6+sw1+w11 ,:3] = therimg1
    img_3[h1:h1+h2, w1+w6+sw1:w1+w6+sw1+w11 ,:3] = therimg2
    img_3[h1+h2:h1+h2+h3, w1+w6+sw1:w1+w6+sw1+w11 ,:3] = therimg3
    img_3[h1+h2+h3:h1+h2+h3+h4, w1+w6+sw1:w1+w6+sw1+w11 ,:3] = therimg4
    img_3[h1+h2+h3+h4:h1+h2+h3+h4+h5, w1+w6+sw1:w1+w6+sw1+w11 ,:3] = therimg5
    img_3[h1+h2+h3+h4+h5:h1+h2+h3+h4+h5+a1h, w1+w6+sw1:w1+w6+sw1+w11 ,:3] = a1
    img_3[:h1, w1+w6+sw1+w11:w1+w6+sw1+w11+w16 ,:3] = therimg6
    img_3[h1:h1+h2, w1+w6+sw1+w11:w1+w6+sw1+w11+w16 ,:3] = therimg7
    img_3[h1+h2:h1+h2+h3, w1+w6+sw1+w11:w1+w6+sw1+w11+w16 ,:3] = therimg8
    img_3[h1+h2+h3:h1+h2+h3+h4, w1+w6+sw1+w11:w1+w6+sw1+w11+w16 ,:3] = therimg9
    img_3[h1+h2+h3+h4:h1+h2+h3+h4+h5, w1+w6+sw1+w11:w1+w6+sw1+w11+w16 ,:3] = therimg10
    img_3[h1+h2+h3+h4+h5:h1+h2+h3+h4+h5+a3h, w1+w6+sw1+w11:w1+w6+sw1+w11+w16 ,:3] = a3
    img_3[h1:h1+h2, :w2,:3] = rgbimg2
    img_3[h1+h2:h1+h2+h3, :w3,:3] = rgbimg3
    img_3[h1+h2+h3:h1+h2+h3+h4, :w2,:3] = rgbimg4
    img_3[h1+h2+h3+h4:h1+h2+h3+h4+h5, :w2,:3] = rgbimg5
    #img_3[h1+h2+h3+h4+h5:h1+h2+h3+h4+h5+a2h, :w2,:3] = a2
    img_3[h1:h1+h2, w1:w1+w7,:3] = rgbimg7
    img_3[h1+h2:h1+h2+h3, w1:w1+w8,:3] = rgbimg8
    img_3[h1+h2+h3:h1+h2+h3+h4, w1:w1+w9,:3] = rgbimg9
    img_3[h1+h2+h3+h4:h1+h2+h3+h4+h5, w1:w1+w10,:3] = rgbimg10
    img_3[:h1+h6, w1+w6+sw1+w11+w16:w1+w6+sw1+w11+w16+iw1,:3] = irgb1
    img_3[h1+h2:h1+h6+ith1, w1+w6+sw1+w11+w16:w1+w6+sw1+w11+w16+iw1,:3] = it1
    img_3[h1+h2+ith1:h1+h6+ith1+meth1, w1+w6+sw1+w11+w16:w1+w6+sw1+w11+w16+iw1,:3] = met1
    # img_3[:h2, w1:w1+w2,:3] = integral[iter]

    # img_3[h1:h1+h3, :w3,:3] = img1
    # img_3[h1:h1+h3, w3:w3+w4,:3] = img2
    # img_3[h1:h1+h3, w2:w3+w4+w5,:3] = img3
    # img_3[h1:h1+h3, w1:w3+w4+w5+w6,:3] = img4
    # img_3[h1:h1+h3, w1+w3:w3+w4+w5+w6+w7,:3] = img5
    # img_3[h1+h3:h1+h3+h4, :w8,:3] = img6
    # img_3[h1+h3:h1+h3+h4, w8:w8+w9,:3] = img7
    # img_3[h1+h3:h1+h3+h4, w8+w7:w8+w9+w10,:3] = img8
    # img_3[h1+h3:h1+h3+h4, w8+w7+w9:w8+w9+w10+w11,:3] = img9
    # img_3[h1+h3:h1+h3+h4, w8+w7+w9+w10:w8+w9+w10+w11+w12,:3] = img10
    # img_3[h2:h2+h13, w1:w1+w13,:3] = img11


    # #cv2.putText(img_3,"Metric:" + str(new_leader_contour), (830,615), cv2.FONT_HERSHEY_SIMPLEX, 1, 255)
    # #cv2.putText(img_3,"Leader_index:" + str(new_leader_index), (820,660), cv2.FONT_HERSHEY_SIMPLEX, 1, 255)

    # # print(img_3.shape)

    # th = 0.1 # defines the value below which a pixel is considered "black"
    # black_pixels = np.where(
    #     (img_3[:, :, 0] < th) & 
    #     (img_3[:, :, 1] < th) & 
    #     (img_3[:, :, 2] < th)
    # )
    # img_3[black_pixels] = [255, 255, 255]

    cv2.imwrite(os.path.join(r'C:\Users\Rakesh\Downloads\Final_result', ('stage_' + str(iter) + '.png')),img_3)
    s = s+1
    o = o+2
    p = p+2
    #person = person
    







In [ ]:


with open(r'C:\Users\Rakesh\Downloads\values.txt', 'a') as f:
        f.write('NumberofDrones'+ ' '+str(NumberofDrones))
        f.write('\n')
        f.write('fov'+ ' '+str(fov))
        f.write('\n')
        f.write('rxthreshold'+ ' '+str(rxthreshold))
        f.write('\n')
        f.write('dem_height'+ ' '+str(dem_height))
        f.write('\n')
        f.write('distance_btwn_drones'+ ' '+str(distance_btwn_drones))
        f.write('\n')
        f.write('Scanning_direction'+ ' '+str(Scanning_direction))
        f.write('\n')
        f.write('Scanning_direction_Waypoint_Distance_c3'+ ' '+str(Scanning_direction_Waypoint_Distance))
        f.write('\n')
        f.write('emptyblobthreshold'+ ' '+str(emptyblobthreshold))
        f.write('\n')
        f.write('changing_to_linear_speed_c5'+ ' '+str(changing_to_linear_speed))
        f.write('\n')
        f.write('local_fac_c1'+ ' '+str(local_fac))
        f.write('\n')
        f.write('global_fac_c2'+ ' '+str(global_fac))
        f.write('\n')
        f.write('minimum_distance_btwn_drone_c4'+ ' '+str(minimum_distance_btwn_drone))
        f.write('\n')
        f.write('person'+ ' '+str(person))
        f.write('\n')
        f.write('blobthreshold'+ ' '+str(emptyblobthreshold))
        f.write('\n')

In [ ]:
import utm
print(utm.from_latlon(48.335836, 14.326644))
print(np.arange(-(int)((10-1)/2),(int)((10+2)/2),1) * 4)

In [ ]:
leader_blob_image_list = []

for imges in sorted(glob.glob(r'C:\Users\Rakesh\Downloads\leader_blob_image' + '/*.png'),key=numericalSort):
    r2= cv2.imread(imges)
    leader_blob_image_list.append(r2)
    
print(leader_blob_image_list)    



In [ ]:
import numpy as np
altitude_list = [35,36,37,38,39,40,41,42,43,44]
print(np.mean(altitude_list))

In [ ]:

currentblob_personposition = [0.5429264720709099, 10.267435782509374] 
dronemeanpos = [0.185841061162089, 0.15672915154027667] 
import numpy as np
import math
angle = np.arctan2(dronemeanpos[1]-currentblob_personposition[1],dronemeanpos[0]-currentblob_personposition[0])
angle = ((np.degrees(angle))%360)+90
print(angle)
if angle == 0 or angle==360 :
    angle = 0                             #need to switch sign of yaxis positive is down and negative is up
    print(angle)


In [ ]:
import numpy as np
coverage = 2*35*np.tan(25)
print(np.tan(np.deg2rad(25)))

In [ ]:
coverage = 2*35*np.tan(np.deg2rad(25))
print(coverage)
rect = [274,401,29,29]
resolution = 512
metretopixels = coverage/resolution
print(metretopixels)
blob_posx = rect[0] + rect[2]/2
blob_posy = rect[1] + rect[3]/2
blob_posxinpixels = blob_posx * metretopixels
blob_posyinpixels = blob_posy * metretopixels
blobposition = [blob_posxinpixels,blob_posyinpixels]
print(blobposition)
integralposition_centre = [-2, 0]
print(integralposition_centre)
integral_startposition = [integralposition_centre[0] - coverage/2, integralposition_centre[1] - coverage/2]
print(integral_startposition)
blob_personposition = [blobposition[0] - abs(integral_startposition[0]), blobposition[1] - abs(integral_startposition[1])]
print(blob_personposition)

In [ ]:
p1 = (0,0)
p2 = (0,10)

# Difference in x coordinates
dx = p2[0] - p1[0]

# Difference in y coordinates
dy = p2[1] - p1[1]

# Angle between p1 and p2 in radians
theta = math.atan2(dy, dx)
print(math.degrees(theta))
print([np.sin(np.deg2rad( 0)), np.cos(np.deg2rad(0))])

In [ ]:
import math
import numpy as np
print((math.sin(np.deg2rad(90))))
print((math.cos(np.deg2rad(90))))

In [ ]:
import math
import numpy as np


person = [22.739795414685666, -4.612425475682947]
new_person= [23.15644494871208, -5.577736082168631]
dx = new_person[0] - person[0]

    # Difference in y coordinates
dy = (-new_person[1] + person[1])

# Angle between p1 and p2 in radians
personorientation = np.arctan2(dy, dx)
personorientation = ((np.degrees(personorientation))%360)+90
print(personorientation)

In [13]:
# import required libraries
import cv2
import numpy as np

# create a black image
img = np.zeros((256, 256, 4), dtype = np.uint8)

# display the image using opencv
#cv2.imshow('black image', img)
#cv2.waitKey(0)

borderoutput = cv2.copyMakeBorder(
    img, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=[0, 0, 255])

borderoutput[:,:,3] = borderoutput[:,:,2] 
 
# showing the image with border


cv2.imwrite(os.path.join(r'C:\Users\Rakesh\Downloads', ('blackupdated.png')),borderoutput)

True